# load libraries

In [ ]:
# load libraries
quiet_library <- function(...) {
    suppressPackageStartupMessages(library(...))
}
quiet_library('tidyverse')
# quiet_library("hise")
quiet_library('ArchR')
quiet_library('data.table')
quiet_library('jsonlite')
quiet_library('parallel')
quiet_library("Seurat")
# quiet_library("Signac")
# Load MOCHA and accompanying libraries
quiet_library('MOCHA')
quiet_library('GenomicRanges')
quiet_library('plyranges')
quiet_library('RMariaDB')
quiet_library('chromVAR')

In [ ]:
packageVersion("MOCHA")

In [ ]:
# # import MP's function to do label transfer from atac to atac
# source('/home/jupyter/github/scATAC_Supplements/scATAC_CellTypeLabeling.R')
suppressPackageStartupMessages(source('/home/jupyter/github/scATAC_Supplements/ArchR_Supplements.R'))
source('/home/jupyter/github/Teaseq-analysis/scRNA_teaseq_ananlysis_helper_functions.r')

In [ ]:
# define work directories
proj_path <- '/home/jupyter/data/preRA_teaseq/EXP-00243'
data_path <- '/home/jupyter/data/preRA_teaseq/EXP-00243'
setwd(proj_path)
# define a project name


In [ ]:
# set ArchR parameters
addArchRThreads(threads = 58)
addArchRGenome("hg38")
set.seed(1221)
options(repr.plot.width = 20, repr.plot.height = 15)

In [ ]:
# define the color palette to be used
npg_color <- c("#E64B35FF", "#4DBBD5FF", "#00A087FF", "#3C5488FF", "#F39B7FFF", 
               "#8491B4FF", "#91D1C2FF", "#DC0000FF", "#7E6148FF", "#B09C85FF")
nejm_color <- c("#BC3C29FF", "#0072B5FF", "#E18727FF", "#20854EFF", "#7876B1FF", "#6F99ADFF", "#FFDC91FF", "#EE4C97FF")
jama_color <- c("#374E55FF", "#DF8F44FF", "#00A1D5FF", "#B24745FF", "#79AF97FF", "#6A6599FF", "#80796BFF")
jco_color <- c("#0073C2FF", "#EFC000FF", "#868686FF", "#CD534CFF", "#7AA6DCFF", "#003C67FF", "#8F7700FF")
cluster_colors <- c("#DC050C", "#FB8072", "#1965B0", "#7BAFDE", "#882E72", "#B17BA6", "#FF7F00", "#FDB462", "#E7298A", 
    "#E78AC3", "#33A02C", "#B2DF8A", "#55A1B1", "#8DD3C7", "#A6761D", "#E6AB02", "#7570B3", "#BEAED4", "#666666", "#999999", 
    "#aa8282", "#d4b7b7", "#8600bf", "#ba5ce3", "#808000", "#aeae5c", "#1e90ff", "#00bfff", "#56ff0d", "#ffff00")
status_colors = c('#5AAA46', '#F59F00', '#840032')
cluster_colors_ext <- colorRampPalette(cluster_colors)(36)
options(repr.plot.width = 20, repr.plot.height = 15)

In [ ]:
# pathway analysis with enrichR
quiet_library(enrichR)
setEnrichrSite("Enrichr")
dbs <- listEnrichrDbs()# pathway analysis with enrichR

In [ ]:
# define work directories
proj_path <- '/home/jupyter/data/preRA_teaseq/EXP-00243/cd4tna_cells'
setwd(proj_path)
# define a project name
proj_name <- 'preRA_tea_seq_cd4tna'
fig_path <- as.character('/home/jupyter/data/preRA_teaseq/EXP-00243/cd4tna_cells/Plots')
output_path <- '/home/jupyter/data/preRA_teaseq/output_results/atac'
if(!dir.exists(fig_path)) (dir.create(fig_path, recursive = TRUE))
if(!dir.exists(output_path)) (dir.create(fig_path, recursive = TRUE))

In [ ]:
#Load scMACS and accompanying libraries
library(MOCHA)
library(data.table)
library(ArchR)
library(GenomicRanges)
library(plyranges)

In [ ]:
# Define your annotation package for TxDb object(s)
# and genome-wide annotation.
# Here our samples are human using hg38 as a reference.
# For more info: <https://bioconductor.org/packages/3.15/data/annotation/>

library(TxDb.Hsapiens.UCSC.hg38.refGene) 
library(org.Hs.eg.db) 
TxDb <- TxDb.Hsapiens.UCSC.hg38.refGene 
Org <- org.Hs.eg.db


In [ ]:
# # helper function

# #### add metadata from seurat object to archr project
# ## extract the columns from seurat to add to archr
# ## add metadata from seurat to archr
# #' @param archr: archr project to add metadata on
# #' @param so: seurat object to extract metadata from
# #' @param so.cols: a list of column names in the seurat obeject
# #' @param id.col: column name of the barcodes which links cells in seurat and archr object, default setting is barcode

# #' @return archr a SummarizedExperiment object which nomarlized counts are aggregrate

# #' @examples 
# # ra_mye_atac <- MetaSotoArchr(archr=ra_mye_atac, so=ra_so_mye, 
# #                            so.cols=c('barcodes', 'subject_id', 'cohort', 
# #                                      'total_pbmc_counts','Tiles_snn_res.0.8', 'wsnn_res.0.8'), 
# #                           id.col='barcodes')

# MetaSotoArchr <- function(archr, so, so.cols, id.col='barcodes', save.archr=TRUE){
#     # check if all the so.cols are in the seurat metadata
#     if(!all(so.cols %in% colnames(so@meta.data))){
#         stop('Not all the columns are present in the seurat object')
#     }
#     if(any(so.cols[so.cols!=id.col] %in% colnames(getCellColData(archr)))){
#         warning('the column names to add are already in the archr object, this will create suffix to the columns')
#     }
#     seurat_metadata <- so@meta.data %>% 
#         select(all_of(c(so.cols, id.col))) %>% as_tibble()
#     # check if id coloum are match between seurat and archr
#     so_ids <- so@meta.data %>% pull(.data[[id.col]]) 
#     archr_ids <- getCellColData(archr) %>% as_tibble() %>% pull(.data[[id.col]]) 
#     if(!setequal(so_ids, archr_ids)){
#         stop('id col in the seurat and archr object dont match.')
#     }
#     # add metadata from seurat to archR
#     cell_id <- archr@cellColData %>% rownames()
#     archr@cellColData <- merge(archr@cellColData, seurat_metadata, by = id.col, sort=FALSE)
#     rownames(archr@cellColData) <- cell_id    
#     if(save.archr){
#         saveArchRProject(ArchRProj = archr)
#     }
#     return(archr)
# }

# ##This function extracts the RNA counts from a Seurat object and adds them to an ArchR Project. 
# ##ATAC_Cell_ID is the metadata column of the Seurat object that contains the matching cell ids to the ArchR Project. Make sure these align before you do anything. 
# ##You can set force to be TRUE or FALSE, depending on whether you want to 
# ##over-write the existing GeneExpressionMatrix within the ArchR Project.
# ##Requires: plyranges, dplyr, Seurat, and ArchR
# addSeuratRNA <- function(SeuratObject, ArchRProject, ATAC_Cell_ID, assay = "RNA", force1 = FALSE){

# 	counts1 <- as.SingleCellExperiment(SeuratObject, assay = assay)
# 	colnames(counts1) = SeuratObject@meta.data[,ATAC_Cell_ID]
# 	genes <- getGenes(ArchRProject) %>% plyranges::filter(symbol %in% as.character(rownames(counts1))) %>% 
# 		arrange(match(symbol,as.character(rownames(counts1))))
# 	counts2 <-counts1[rownames(counts1) %in% genes$symbol]
# 	counts3 <- SummarizedExperiment(assays=list(counts=assays(counts2)[[1]]),
# 		 rowRanges= genes) 
# 	ArchRProject <- addGeneExpressionMatrix(ArchRProject,
# 					seRNA = counts3, force = force1)
# 	return(ArchRProject)
# }

# load cd4 naive t cells atac data

In [ ]:
# load ATAC 
# load the ArchR project for CD4 t cells
cd4na_atac <- loadArchRProject(path = '/home/jupyter/data/preRA_teaseq/EXP-00243/cd4tna_cells')
cd4na_atac

In [ ]:
# cd4na_atac <- addImputeWeights(cd4na_atac)

In [ ]:
# load the seurat object
ra_tea_cd4na <- readRDS(file.path(data_path, 'PreRA_teaseq_seurat_cd4naive_t_cells_06112023.rds'))

In [ ]:
ra_tea_cd4na

In [ ]:
# ra_tea_cd4na@meta.data %>% colnames()
kit_id=str_split(ra_tea_cd4na@meta.data %>% distinct(pbmc_sample_id) %>% pull(pbmc_sample_id),
               pattern = '-', simplify = TRUE)[,1] %>% str_replace('PB', 'KT')
kit_id %>% paste(collapse = ', ')

In [ ]:
ra_tea_cd4na@meta.data %>% distinct(subject_id) %>% pull(subject_id)

In [ ]:
# check all the atac barcodes matched
ra_tea_cd4na@meta.data %>% colnames()
all(ra_tea_cd4na@meta.data$atac_cell_id %in% cd4na_atac$cellNames)
all(cd4na_atac$cellNames %in% ra_tea_cd4na@meta.data$atac_cell_id)

In [ ]:
# extracts the RNA counts from a Seurat object and adds them to an ArchR Project
cd4na_atac <- addSeuratRNA(SeuratObject = ra_tea_cd4na,ArchRProject =  cd4na_atac,
                           'atac_cell_id',  assay = "SCT")

In [ ]:
getAvailableMatrices(cd4na_atac)

In [ ]:
# cd4na_atac
# ra_tea_cd4na

In [ ]:
metacols <- c('barcodes', 'predicted.celltype_l2', 
              'wsnn_res.0.3','age', 'total_counts')
metacols %in%  colnames(ra_tea_cd4na@meta.data)
metacols %in%  colnames(getCellColData(cd4na_atac))

In [ ]:
# add some metadate from seurat to archR
getCellColData(cd4na_atac) %>% colnames()
cd4na_atac <- MetaSotoArchr(archr=cd4na_atac, so=ra_tea_cd4na, 
                           so.cols=c('barcodes', 
                                     'predicted.celltype_l2',
                                     'wsnn_res.0.3','age', 'total_counts'), 
                          id.col='barcodes')

In [ ]:
getCellColData(cd4na_atac) %>% colnames()

In [ ]:
cd4na_atac <- addClusters(
      input = cd4na_atac,
      reducedDims = "IterativeLSI",
      method = "Seurat",
      name = "Clusters_0.5",
      resolution = 0.5,
      force = TRUE
    )

In [ ]:
saveArchRProject(ArchRProj = cd4na_atac)

In [ ]:
# add archr clusters into seurat
atac_clusters_0.5 <- getCellColData(cd4na_atac) %>% as_tibble() %>% 
      dplyr::select(Clusters_0.5, barcodes) %>% dplyr::rename('archr_clusters_0.5'='Clusters_0.5')
cell_id <- ra_tea_cd4na@meta.data %>% rownames()
ra_tea_cd4na@meta.data <- ra_tea_cd4na@meta.data %>% left_join(atac_clusters_0.5, by = 'barcodes')
rownames(ra_tea_cd4na@meta.data) <- cell_id

In [ ]:
# load the seurat object
ra_tea_cd4na %>% saveRDS(file.path(data_path, 'PreRA_teaseq_seurat_cd4naive_t_cells_06112023.rds'))

In [ ]:
ra_tea_cd4na@meta.data %>% colnames()

In [ ]:
getAvailableMatrices(cd4na_atac)
getCellColData(cd4na_atac) %>% colnames()

In [ ]:
# cd4t_atac_gene_cores <- getMatrixFromProject(cd4t_atac, useMatrix = "GeneScoreMatrix")

In [ ]:
# check the differential markers in atac clusters
markersGS <- getMarkerFeatures(
    ArchRProj = cd4na_atac, 
    useMatrix = "GeneScoreMatrix", 
    groupBy = "Clusters_0.5",
    bias = c("TSSEnrichment", "log10(nFrags)"),
    testMethod = "wilcoxon"
)

In [ ]:
markerList <- getMarkers(markersGS, cutOff = "FDR <= 0.01 & Log2FC >= 0.5")
markerList$C4$name

In [ ]:
# marker_genes <- rowData(cd4t_atac_gene_cores)$name %>% str_subset('GATA3|RORC|FOXP3|TBX21|IKZF2|STAT3')
marker_genes <- c('IL21')
marker_genes
p2 <-  plotEmbedding(
    ArchRProj = cd4na_atac, 
    colorBy = "GeneScoreMatrix", 
    name = marker_genes, 
    embedding = "UMAP",
    imputeWeights = getImputeWeights(cd4na_atac)
)
p2
plotPDF(p2, name = paste0(proj_name, '_IL21_atac_umap_gene_score.pdf'), ArchRProj = cd4na_atac,
        addDOC = FALSE, width = 6, height = 6)

In [ ]:
# marker_genes <- rowData(cd4t_atac_gene_cores)$name %>% str_subset('GATA3|RORC|FOXP3|TBX21|IKZF2|STAT3')
marker_genes <- c('GATA3', 'RORC', 'FOXP3', 'TBX21', 'IKZF2', 'STAT3', 'SELL', 'IL17A', 'IFNG', 'IL10')
marker_genes
p2 <-  plotEmbedding(
    ArchRProj = cd4na_atac, 
    colorBy = "GeneScoreMatrix", 
    name = marker_genes, 
    embedding = "UMAP",
    imputeWeights = getImputeWeights(cd4na_atac)
)

In [ ]:
# marker_genes <- rowData(cd4t_atac_gene_cores)$name %>% str_subset('GATA3|RORC|FOXP3|TBX21|IKZF2|STAT3')
marker_genes <- c('IL6R', 'IL4R', 'IL13RA1', 'IL6ST')
marker_genes
p2 <-  plotEmbedding(
    ArchRProj = cd4na_atac, 
    colorBy = "GeneScoreMatrix", 
    name = marker_genes, 
    embedding = "UMAP",
    imputeWeights = getImputeWeights(cd4na_atac)
)
plotPDF(p2, name = paste0(proj_name, '_IL6R_atac_umap_gene_score.pdf'), ArchRProj = cd4na_atac,
        addDOC = FALSE, width = 6, height = 6)     

In [ ]:
p2

In [ ]:
# plot umap of annotated cell types
p1 <- plotEmbedding(cd4na_atac, embedding = "UMAP", 
                    colorBy = "cellColData", name = c('subject_id'))
plotPDF(p1, name = paste0(proj_name, '_subject_id_atac_umap_gene_score.pdf'), ArchRProj = cd4na_atac,
        addDOC = FALSE, width = 6, height = 6)                                                

In [ ]:
plotEmbedding(cd4na_atac, embedding = "UMAP", 
                    colorBy = "GeneScoreMatrix", name = c('TGFBR1'))

In [ ]:
# plot umap of annotated cell types
p1 <- plotEmbedding(cd4na_atac, embedding = "UMAP", 
                    colorBy = "cellColData", name = c("l2_cell_types", "Clusters_0.5", 'subject_id','age',
                                                      'predicted.celltype_l2','wsnn_res.0.5',
                                                      'cohort'))
p2 <-  plotEmbedding(
    ArchRProj = cd4na_atac, 
    colorBy = "GeneScoreMatrix", 
    name = marker_genes, 
    embedding = "UMAP",
    imputeWeights = getImputeWeights(cd4na_atac)
)

plotPDF(p1,p2, name = paste0(proj_name, '_Clusters_0.8_atac_umap_gene_score.pdf'), ArchRProj = cd4na_atac,
        addDOC = FALSE, width = 6, height = 6)

In [ ]:
# plot umap of annotated cell types
p1 <- plotEmbedding(cd4na_atac, embedding = "UMAP", 
                    colorBy = "cellColData", name = c("Clusters_0.5",'cohort', 'subject_id'
                                                      ))
p1
plotPDF(p1, name = paste0(proj_name, '_Clusters_0.5_atac_umap.pdf'), ArchRProj = cd4na_atac,
        addDOC = FALSE, width = 6, height = 6)

In [ ]:
cd4na_atac$cohort=if_else(cd4na_atac$cohort=='pre-RA', 'At-risk', cd4na_atac$cohort)

In [ ]:
# plot umap of annotated cell types
p1 <- plotEmbedding(cd4na_atac, embedding = "UMAP",  colorBy = "cellColData", name = c("cohort"))
p1 <- p1 + theme(legend.text = element_text(size=50)) +
  guides(color = guide_legend(override.aes = list(size = 30)))
p1 
plotPDF(p1, name = paste0(proj_name, '_cohort_atac_umap.pdf'), ArchRProj = cd4na_atac,
        addDOC = FALSE, width = 6, height = 6)

In [ ]:
cd4na_atac$Sex %>% unique()

In [ ]:
getCellColData(cd4na_atac) %>% as_tibble() %>% colnames()
Clusters_0.5_counts <- getCellColData(cd4na_atac) %>% as_tibble() %>% 
    group_by(Clusters_0.5, subject_id, cohort) %>% tally() %>%
    ungroup() %>% group_by(subject_id) %>% 
    mutate(total_count=sum(n), frequency=n/total_count) 
Clusters_0.5_counts %>% ggpubr::ggboxplot(x='cohort', y='frequency', 
                                  color='cohort', add = "jitter",size = 0.3,
              add.params = list(size = 0.6))+
    ylab('Frequency')+
    facet_wrap(vars(Clusters_0.5), scales = 'free_y') + xlab('')+
    scale_color_manual( values = c('#0072B5FF', '#BC3C29FF')) + 
    ggpubr::stat_compare_means(method = "wilcox.test", label.x.npc=0.3)
ggsave(file.path(fig_path, paste0(proj_name, 
                                  '_atac_clusters_0.5_frequency_boxplot.png')), 
       width=12, height=8)

In [ ]:
# run wilcox test for frequency pretreatment vs healthy 
freq_stats_wilcox_pre <- Clusters_0.5_counts %>% 
    group_by(Clusters_0.5) %>% 
    rstatix::wilcox_test(data =., frequency ~ cohort, paired = FALSE) %>%
      rstatix::adjust_pvalue(method = "BH") %>%
      rstatix::add_significance("p.adj") 
freq_stats_wilcox_pre %>% arrange(p)

## run differential gene scores based on archr

In [ ]:
getCellColData(cd4na_atac) %>% as_tibble() %>% group_by(cohort) %>% tally()

In [ ]:
# get differential gene score between preRA vs healthy
markersGS <- getMarkerFeatures(
    ArchRProj = cd4na_atac, 
    useMatrix = "GeneScoreMatrix", 
    groupBy = "cohort",
    useGroups = 'pre-RA',
  bgdGroups = 'Healthy',
    maxCells = 12765,
    bias = c("TSSEnrichment", "log10(nFrags)"),
    testMethod = "wilcoxon"
)

In [ ]:
# get all the gene scores tested in cd4 naive
markerList <- getMarkers(
  seMarker = markersGS,
  cutOff = "NA",
  n = NULL,
  returnGR = FALSE
)


### compare differntial gene scores and differential gene expression

In [ ]:
# load the degs list for cd4 naive
cd4na_pseudo_degs <- read_csv('/home/jupyter/data/preRA_teaseq/output_results/cd4_t/PreRA_teaseq_cd4_t_t_anno_atac_cd4naive_PseudoBulk_degs_preRA.csv',
                             show_col_types = FALSE) %>% 
    mutate('log10FDR'=if_else(logFC>0, -log10(FDR), log10(FDR))) 
cd4na_pseudo_degs %>% head()

In [ ]:
cd4na_gene_scores <- markerList$`pre-RA` %>% as_tibble()
cd4na_gene_scores <- cd4na_gene_scores %>% 
    mutate('log10FDR'=if_else(Log2FC>0, -log10(FDR), log10(FDR))) %>%
    dplyr::rename('gene'='name') 
cd4na_gene_scores %>% filter(FDR<0.05&Log2FC>0.1)

In [ ]:
cd4na_gene_scores %>% filter(str_detect(gene, 'TGFBR'))

In [ ]:
table(cd4na_gene_scores$gene %in% cd4na_pseudo_degs$gene)
table(cd4na_pseudo_degs$gene %in% cd4na_gene_scores$gene)

In [ ]:
# join two list
cd4na_degs_gene_scores <- cd4na_pseudo_degs %>%
    full_join(cd4na_gene_scores, by='gene', suffix = c('_rna', '_atac'))%>%
    mutate(log10FDR_rna=replace_na(log10FDR_rna, 0), log10FDR_atac=replace_na(log10FDR_atac, 0)) %>%
    mutate(deg_gs_cat = case_when(log10FDR_rna>log10(0.05) & log10FDR_atac>log10(0.05) ~ 'SigdeggsUp'))
# cd4na_degs_gene_scores %>% filter(is.na(FDR_atac)|is.na(FDR_rna))

In [ ]:
cd4na_degs_gene_scores %>% colnames()

In [ ]:
# only take the overlap genes
cd4na_degs_gene_scores_fl <- cd4na_degs_gene_scores %>% 
    mutate(sig_deg=FDR_rna<0.05,
          sig_genescores=FDR_atac<0.05&abs(Log2FC)>0.1,
          poised_status=case_when(sig_deg & sig_genescores ~ 'atac change & rna change',
                                 !sig_deg & sig_genescores ~ 'atac change & rna no change',
                                 sig_deg & !sig_genescores ~ 'atac no change & rna change',
                                 !sig_deg & !sig_genescores ~ 'atac no change & rna no change')) %>% 
    filter(!is.na(FDR_rna)&!is.na(FDR_atac))

In [ ]:
cd4na_degs_gene_scores_fl %>% distinct(gene, .keep_all = TRUE) %>% 
    group_by(poised_status) %>% tally()

In [ ]:
# plot the gene scores and 
cd4na_degs_gene_scores_fl %>% filter(poised_status!='atac no change & rna no change') %>%
    ggplot(aes(x=logFC, y=Log2FC, label = gene, color=poised_status))+
    geom_point() + ggrepel::geom_text_repel(box.padding = 0.5, max.overlaps = 5) +
    labs(x='logFC DEGs', 
        y='LogFC Gene scores') 
# ggsave(file.path(fig_path, paste0(proj_name, '_gene_exp_promotor_gene_score_promoters.pdf')), width = 6, height = 6)

In [ ]:
# plot the gene scores and 
cd4na_degs_gene_scores_fl %>% filter(poised_status!='atac no change & rna no change')%>% 
    ggplot(aes(x=log10FDR_rna, y=log10FDR_atac, label = gene, color=poised_status))+
    geom_point() + 
    geom_hline(yintercept=-log10(0.05), linetype="dashed", color = "red")+
    geom_hline(yintercept=log10(0.05), linetype="dashed", color = "red") +
    geom_vline(xintercept=-log10(0.05), linetype="dashed", color = "red") +
    geom_vline(xintercept=log10(0.05), linetype="dashed", color = "red") +
    ggrepel::geom_text_repel(box.padding = 0.5, max.overlaps = 5) +
    labs(x='log10(FDR) DEGs', 
        y='log10(FDR) Gene scores') + theme_bw()+ theme(legend.position = "none")
ggsave(file.path(fig_path, paste0(proj_name, '_gene_exp_promotor_gene_score_promoters.png')), width = 12, height = 12)

In [ ]:
cd4na_degs_gene_scores_fl %>% filter(poised_status=='atac change & rna no change' & 
                                     Log2FC<0) %>% nrow()

In [ ]:
# gene enrichment 
poised_gene_up <- cd4na_degs_gene_scores_fl %>% 
    filter(FDR_atac<0.05&FDR_rna>0.05&Log2FC>0.1) %>% pull(gene) %>% unique() 
poised_gene_up %>% length()
# pathway enrichment for cd4na
enrich_cd4na_posied_up <- enrichr(poised_gene_up, c('KEGG_2021_Human', 'MSigDB_Hallmark_2020'))


In [ ]:
cd4na_degs_gene_scores_fl %>% filter(str_detect(gene, 'TGFB'))

In [ ]:
enrich_cd4na_posied_up[["MSigDB_Hallmark_2020"]]%>% filter(Adjusted.P.value<0.05) %>% 
    dplyr::select(-c(Old.P.value, Old.Adjusted.P.value, P.value, Combined.Score	))

In [ ]:
# gene enrichment 
poised_gene_down <- cd4na_degs_gene_scores_fl %>% 
    filter(FDR_atac<0.05&FDR_rna>0.05&Log2FC < -0.1) %>% pull(gene) %>% unique() 
poised_gene_down %>% length()
# pathway enrichment for cd4na
enrich_cd4na_posied_down <- enrichr(poised_gene_down, c('KEGG_2021_Human', 'MSigDB_Hallmark_2020'))
 

In [ ]:
enrich_cd4na_posied_down[["MSigDB_Hallmark_2020"]]%>% filter(Adjusted.P.value<0.05) %>% 
    dplyr::select(-c(Old.P.value, Old.Adjusted.P.value))

In [ ]:
# gene enrichment 
ra_gene <- cd4na_degs_gene_scores_fl %>% 
    filter(FDR_rna<0.05&Log2FC>0) %>% pull(gene) %>% unique() 
ra_gene %>%length()
# pathway enrichment for cd4na
enrich_cd4na_up <- enrichr(ra_gene, c('KEGG_2021_Human', 'MSigDB_Hallmark_2020'))


In [ ]:
enrich_cd4na_up[["MSigDB_Hallmark_2020"]]%>% filter(Adjusted.P.value<0.05) %>% 
    dplyr::select(-c(Old.P.value, Old.Adjusted.P.value))

### check gene scores and gene difference for C4 and C5

In [ ]:
cd4na_atac$Clusters_0.8

In [ ]:
cd4na_atac$Clusters_0.5_anno <- if_else(cd4na_atac$Clusters_0.5 %in% c('C4', 'C5'), 'preRA_clusters', 'healthy_clusters')

In [ ]:
cd4na_atac$Clusters_0.5_anno %>% unique()
cd4na_atac$Clusters_0.5_anno %>% table()

In [ ]:
# get differential gene score between preRA vs healthy
markersC4_C5 <- getMarkerFeatures(
    ArchRProj = cd4na_atac, 
    useMatrix = "GeneScoreMatrix",
    groupBy = "Clusters_0.5_anno",
    useGroups = c('preRA_clusters'),
  bgdGroups = c('healthy_clusters'),
    maxCells = 5000,
    bias = c("TSSEnrichment", "log10(nFrags)"),
    testMethod = "wilcoxon"
)

In [ ]:
# get all the gene scores tested in cd4 naive
markerList <- getMarkers(
  seMarker = markersC4_C5,
  cutOff = "FDR<10",
  n = NULL,
  returnGR = FALSE
)


In [ ]:
markerList$preRA_clusters

In [ ]:
markerList$preRA_clusters %>% as_tibble() %>% ggplot(aes(x=FDR)) +geom_histogram()

In [ ]:
# convert the gene score into tibble
c4c5_gene_scores <- markerList$preRA_clusters %>% as_tibble()
c4c5_gene_scores <- c4c5_gene_scores %>% 
    mutate('log10FDR'=if_else(Log2FC>0, -log10(FDR), log10(FDR))) %>%
    dplyr::rename('gene'='name') 
c4c5_gene_scores %>% filter(FDR<0.05&Log2FC>0.1) %>% nrow()

In [ ]:
# load deg betweet C4 and C5
c4_c5_degs <- read_csv('/home/jupyter/data/preRA_teaseq/output_results/cd4na_t/PreRA_teaseq_cd4na_tarchr_clusters_0.5C4_C5_RAvsHealthy_DEGs.csv',
                      show_col_types = FALSE)%>% 
    mutate('log10FDR'=if_else(avg_log2FC>0, -log10(p_val_adj), log10(p_val_adj))) 
c4_c5_degs %>% head()

In [ ]:
# join two list
c4_c5_degs_gene_scores <- c4_c5_degs %>%
    full_join(c4c5_gene_scores, by='gene', suffix = c('_rna', '_atac'))%>%
    mutate(log10FDR_rna=replace_na(log10FDR_rna, 0), log10FDR_atac=replace_na(log10FDR_atac, 0)) %>%
    mutate(deg_gs_cat = case_when(log10FDR_rna>log10(0.05) & log10FDR_atac>log10(0.05) ~ 'SigdeggsUp'))
# cd4na_degs_gene_scores %>% filter(is.na(FDR_atac)|is.na(FDR_rna))

In [ ]:
c4_c5_degs_gene_scores %>% colnames()

In [ ]:
# only take the overlap genes
c4_c5_degs_gene_scores_fl <- c4_c5_degs_gene_scores %>% 
    mutate(sig_deg=p_val_adj<0.05&abs(avg_log2FC)>0.1,
          sig_genescores=FDR<0.05&abs(Log2FC)>0.5,
          poised_status=case_when(sig_deg & sig_genescores ~ 'atac change & rna change',
                                 !sig_deg & sig_genescores ~ 'atac change & rna no change',
                                 sig_deg & !sig_genescores ~ 'atac no change & rna change',
                                 !sig_deg & !sig_genescores ~ 'atac no change & rna no change'))  %>% 
    filter(!is.na(p_val_adj)&!is.na(FDR)) %>% filter()

In [ ]:
c4_c5_degs_gene_scores_fl %>% 
    group_by(poised_status) %>% tally()

In [ ]:
c4_c5_degs_gene_scores_fl %>% filter(log10FDR_rna==0)

In [ ]:
# plot the gene scores and 
c4_c5_degs_gene_scores_fl %>% filter(poised_status!='atac no change & rna no change') %>%
    ggplot(aes(x=avg_log2FC, y=Log2FC, label = gene, color=poised_status))+
    geom_point() + ggrepel::geom_text_repel(box.padding = 0.5, max.overlaps = 5) +
    labs(x='logFC DEGs', 
        y='LogFC Gene scores') + theme_bw()+ theme(legend.position = "none")
ggsave(file.path(fig_path, paste0(proj_name, 'c4_c5_gene_exp_promotor_gene_score_promoters.pdf')), width = 6, height = 6)

In [ ]:
# plot the gene scores and 
c4_c5_degs_gene_scores_fl %>% filter(poised_status!='atac no change & rna no change')%>% 
    ggplot(aes(x=log10FDR_rna, y=log10FDR_atac, label = gene, color=poised_status))+
    geom_point() + 
    geom_hline(yintercept=-log10(0.05), linetype="dashed", color = "red")+
    geom_hline(yintercept=log10(0.05), linetype="dashed", color = "red") +
    geom_vline(xintercept=-log10(0.05), linetype="dashed", color = "red") +
    geom_vline(xintercept=log10(0.05), linetype="dashed", color = "red") +
    xlim(-20, 20)+
    ggrepel::geom_text_repel(box.padding = 0.5, max.overlaps = 5) + 
    labs(x='log10(FDR) DEGs', 
        y='log10(FDR) Gene scores') + theme_bw()+ theme(legend.position = "none")
ggsave(file.path(fig_path, paste0(proj_name, 'c4_c5_gene_exp_promotor_gene_score_promoters.png')), width = 12, height = 12)

In [ ]:
# gene enrichment 
poised_gene_up <- c4_c5_degs_gene_scores_fl %>% 
    filter(FDR<0.05&p_val_adj>0.2&Log2FC>0.1) %>% pull(gene) %>% unique() 
poised_gene_up %>% length()
# pathway enrichment for cd4na
enrich_c4c5_posied_up <- enrichr(poised_gene_up, c('KEGG_2021_Human', 'MSigDB_Hallmark_2020'))


In [ ]:
enrich_c4c5_posied_up[["MSigDB_Hallmark_2020"]]%>% filter(Adjusted.P.value<0.05) %>% 
    dplyr::select(-c(Old.P.value, Old.Adjusted.P.value))

## call peaks in MOCHA and run differential

In [ ]:
getCellColData(cd4na_atac) %>% as_tibble() %>% 
    group_by( subject_id) %>% tally()

In [ ]:
getCellColData(cd4na_atac) %>% colnames()

In [ ]:
cd4na_atac$cd4t_annotation = 'CD4 Naive'

In [ ]:
getCellColData(cd4na_atac) %>% colnames()

In [ ]:
# # set up the parameters to call peaks 
# # Parameters for calling open tiles
# cellPopulations <- cd4na_atac$Clusters_0.5 %>% unique()
# cellPopLabel <- "Clusters_0.5"
# numCores=60


In [ ]:
cd4na_atac$clean_l2_cell_types='cd4_naive'

In [ ]:
# # call peaks in total CD4 naive cells
cellPopulations <- cd4na_atac$clean_l2_cell_types %>% unique()
cellPopLabel <- "clean_l2_cell_types"
numCores=55
groupColumn <- "cohort" 
join <- "union"

In [ ]:
cd4na_atac$clean_l2_cell_types %>% unique()

In [ ]:
# getting studySignal from the archr project that has all the cells

# load archR data
ra_atac <- loadArchRProject(path = "/home/jupyter/data/preRA_teaseq/EXP-00243/atac_arrows")
studySignal = median(ra_atac@cellColData$nFrags)
studySignal

In [ ]:
getCellColData(cd4na_atac) %>% colnames() %>% sort()

In [ ]:
packageVersion('MOCHA')

In [ ]:
####################################################
# 2. Call open tiles (main peak calling step)
#    Done once for all specified cell populations
####################################################
cd4na_tileResults <- MOCHA::callOpenTiles(
    cd4na_atac,  
   studySignal = studySignal,
    cellPopLabel = 'clean_l2_cell_types',
    cellPopulations = 'cd4_naive',
    numCores = numCores, 
    outDir = '/home/jupyter/data/preRA_teaseq/EXP-00243/cd4tna_cells/MOCHA',
    TxDb = 'TxDb.Hsapiens.UCSC.hg38.refGene', 
    OrgDb = 'org.Hs.eg.db'
)


In [ ]:
# load the tile matrix from scmacs
cd4na_tileResults %>% saveRDS(file.path(output_path, 
                                  paste0(proj_name, '_MOCHA_atac_total_cd4na_tiles_matrix.rds')))

In [ ]:
# # save thetile matrix from scmacs
# cd4na_tileResults %>% saveRDS(file.path(data_path, 
#                                   paste0(proj_name, '_MOCHA_atac_total_cd4na_tiles_matrix.rds')))

In [ ]:
# Parameters for downstream analysis
# cellPopulation <- "cd4_naive"
threshold <- 0.2
groupColumn <- "cohort"
join <- "union"

In [ ]:
####################################################
# 3. Get reproducible sample-peak matrix
#    Done for each cell population individually
####################################################

# Parameters for downstream analysis
# cellPopulation <- "cd4_naive"
threshold <- 0.2
groupColumn <- "cohort"
join <- "union"

SampleTileMatrices <- MOCHA::getSampleTileMatrix( 
    cd4na_tileResults,
    cellPopulations = cellPopulations,
    groupColumn = groupColumn,
    threshold = threshold
)


In [ ]:
SampleTileMatrices

In [ ]:
####################################################
# 4. Add gene annotations to our SampleTileMatrices,
#    labelling tiles as either a promoter, exonic,
#    intronic, or distal region. Gene names are 
#    given for all but distal. This info will aid 
#    further downstream analyses but is not required 
#    for differential accessibility.
#    This function can also take any GRanges object
#    and add annotations to its metadata.
####################################################
SampleTileMatricesAnnotated <- MOCHA::annotateTiles( 
  SampleTileMatrices
)

In [ ]:
# metadata(SampleTileMatricesAnnotated)

In [ ]:
# SampleTileMatricesAnnotated <- scMACS::annotateTiles( 
#   SampleTileMatricesAnnotated
# )

In [ ]:
# Load a curated motif set from library(chromVARmotifs) 
# included with ArchR installation
library(chromVARmotifs)
# library(TFBSTools)
# data("human_pwms_v2")
data('human_pwms_v2')
SampleTileMatricesAnnotated <- MOCHA::addMotifSet(motifPWMs = human_pwms_v2,
  SampleTileMatricesAnnotated, 
  # pwms = human_pwms_v2,  
  w = 7 # width parameter for motifmatchr::matchMotifs()
)

In [ ]:
SampleTileMatricesAnnotated

In [ ]:
metadata(SampleTileMatricesAnnotated)$Motifs$TFAP2B$score %>% hist()

In [ ]:
SampleTileMatricesAnnotated %>% saveRDS(file.path(output_path, 
                                  paste0(proj_name, '_MOCHA_total_cd4na_SampleTileMatrices.rds')))

In [ ]:
# # # load the sample tile matrix
# SampleTileMatrices <- readRDS(file.path(data_path, 
#                                   paste0(proj_name, '_scMACS_atac_Clusters_0.5_sampleTileMatrixs.rds')))

In [ ]:
# load the sample tile matrix
SampleTileMatricesAnnotated <- readRDS(file.path(output_path,
                                        paste0(proj_name, '_MOCHA_total_cd4na_SampleTileMatrices.rds')))

In [ ]:
SampleTileMatricesAnnotated

### import peaks called by mocha into archr

In [ ]:
# make a grange object from the the union of the peaks called
granges_mocha <- SampleTileMatricesAnnotated %>% rownames() %>% unique() %>% 
    StringsToGRanges()
granges_mocha

In [ ]:
rowData(SampleTileMatricesAnnotated)$cd4_naive %>% unique()

In [ ]:
## Create new feature matrix for mocha call peaks
cd4na_atac <- addFeatureMatrix(cd4na_atac, matrixName = "MochaPeakMatrix",
                               features = granges_mocha, binarize = FALSE)

In [ ]:
## Create new feature matrix for mocha call peaks
cd4na_atac <- addPeakSet(cd4na_atac, peakSet = granges_mocha, force = TRUE)

In [ ]:
## Create new feature matrix for mocha call peaks
cd4na_atac <- addPeakMatrix(
  ArchRProj = cd4na_atac,
  ceiling = 4,
  binarize = FALSE,
  verbose = TRUE,
  threads = getArchRThreads(),
  parallelParam = NULL,
  force = TRUE,
  logFile = createLogFile("addPeakMatrix")
)


In [ ]:
cd4na_atac <- addMotifAnnotations(ArchRProj = cd4na_atac, motifSet = "cisbp", name = "Motif",
                                 force = TRUE)

In [ ]:
cd4na_atac <- addBgdPeaks(cd4na_atac, force = TRUE)

In [ ]:
cd4na_atac <- addDeviationsMatrix(
    matrixName = 'ChromvarMatrix',
  ArchRProj = cd4na_atac, 
  peakAnnotation = "Motif",
  force = TRUE
)

In [ ]:
getAvailableMatrices(cd4na_atac)

In [ ]:
plotVarDev <- getVarDeviations(cd4na_atac, name = "ChromvarMatrix", plot = TRUE)
plotVarDev

In [ ]:
markerTFs <- getFeatures(cd4na_atac, select = paste(tfs, collapse="|"), useMatrix = "MotifMatrix")
markerTFs <- sort(grep("z:", markerTFs, value = TRUE))
TFnames <- stringr::str_split(stringr::str_split(markerTFs, 
                                                 pattern = "\\.", simplify=TRUE)[,2],
                              pattern = "-", simplify = TRUE)[,1]
markerTFs <- markerTFs[!duplicated(TFnames)]
markerTFs

In [ ]:
saveArchRProject(ArchRProj = cd4na_atac)

In [ ]:
getAvailableMatrices(cd4na_atac)
table(cd4na_atac$Clusters_0.5)
table(cd4na_atac$Clusters_0.5_anno)

In [ ]:
# get differential gene score between preRA vs healthy
markersC4_C5 <- getMarkerFeatures(
    ArchRProj = cd4na_atac, 
    useMatrix = "PeakMatrix", 
    groupBy = "Clusters_0.5_anno",
    useGroups = c('preRA_clusters'),
  bgdGroups = c('healthy_clusters'),
    maxCells = 5000,
    bias = c("TSSEnrichment", "log10(nFrags)"),
    testMethod = "wilcoxon"
)

In [ ]:
metadata(markersC4_C5)$Params$useMatrix <- 'PeakMatrix'

In [ ]:
pma <- plotMarkers(seMarker = markersC4_C5, name = "preRA_clusters",
                   cutOff = "FDR <= 0.05 & abs(Log2FC) >= 0.5", plotAs = "MA")
pma
plotPDF(pma, name = paste0(proj_name, '_c4c5_vs_rest_DAPs_MAplot.pdf'), ArchRProj = cd4na_atac,
        addDOC = FALSE, width = 4, height = 4)

In [ ]:
pma
plotPDF(pma, name = paste0(proj_name, '_c4c5_vs_rest_DAPs_MAplot.pdf'), ArchRProj = cd4na_atac,
        addDOC = FALSE, width = 4, height = 4)

In [ ]:
markerList <- getMarkers(markersC4_C5, cutOff = "FDR <= 0.05 & abs(Log2FC) >= 0.5", returnGR = TRUE)


In [ ]:
markerList

In [ ]:
pv <- plotMarkers(seMarker = markersC4_C5, name = "preRA_clusters",
                 cutOff = "FDR <= 0.1 & abs(Log2FC) >= 1", plotAs = "Volcano")
pv

In [ ]:
motifsUp <- peakAnnoEnrichment(
    seMarker = markersC4_C5,
    ArchRProj = cd4na_atac,
    peakAnnotation = "Motif",
    cutOff = "FDR <= 0.1 & Log2FC >= 0.5"
  )

In [ ]:
df <- data.frame(TF = rownames(motifsUp), mlog10Padj = assay(motifsUp)[,1])
df <- df[order(df$mlog10Padj, decreasing = TRUE),]
df$rank <- seq_len(nrow(df))
ggUp <- ggplot(df, aes(rank, mlog10Padj, color = mlog10Padj)) + 
  geom_point(size = 1) +
  ggrepel::geom_label_repel(
        data = df[rev(seq_len(30)), ], aes(x = rank, y = mlog10Padj, label = TF), 
        size = 5,
        nudge_x = 2,
        color = "black"
  ) + theme_ArchR() + 
  ylab("-log10(P-adj) Motif Enrichment") + 
  xlab("Rank Sorted TFs Enriched") +
  scale_color_gradientn(colors = paletteContinuous(set = "comet"))

ggUp

In [ ]:
# subset the peaks with in the promoter enhancer database
subsetByOverlaps(prom_enh_cd4na_enh, markerList$preRA_clusters) %>% as_tibble() %>% pull(Gene) %>% sort()


In [ ]:
subsetByOverlaps(prom_enh_cd4na_enh, markerList$preRA_clusters) %>% as_tibble() %>% pull(Gene) %>% str_subset('C')

In [ ]:
il6r_granges
cd4na_diff_sig_il <- subsetByOverlaps(cd4na_diff_sig, il6r_granges)
cd4na_diff_sig_il

In [ ]:
getAvailableMatrices(cd4na_atac)

In [ ]:
p <- plotBrowserTrack(
    ArchRProj = cd4na_atac, 
    groupBy = "Clusters_0.5", 
    geneSymbol = c("IL6R", 'IL4R', 'IL13RA1' , 'IL21R','IL21', 'IFNAR1'),
    features =  markerList$preRA_clusters,
    useMatrix='GeneScoreMatrix',
     tileSize = 500,
  #minCells = 500,
    upstream = 50000,
    downstream = 50000
)

In [ ]:
# marker_genes <- rowData(cd4t_atac_gene_cores)$name %>% str_subset('GATA3|RORC|FOXP3|TBX21|IKZF2|STAT3')
marker_genes <- c('IL21')
marker_genes
p2 <-  plotEmbedding(
    ArchRProj = cd4na_atac, 
    colorBy = "GeneScoreMatrix", 
    name = marker_genes, 
    embedding = "UMAP",
    imputeWeights = getImputeWeights(cd4na_atac)
)
p2
p3 <-  plotEmbedding(
    ArchRProj = cd4na_atac, 
    colorBy = "GeneExpressionMatrix", 
    name = marker_genes, 
    embedding = "UMAP",
    imputeWeights = NULL
)
p3
plotPDF(p2, p3, name = paste0(proj_name, '_il21_expression_promoter.pdf'), ArchRProj = cd4na_atac,
        addDOC = FALSE, width = 4, height = 4)

In [ ]:
getAvailableMatrices(cd4na_atac)


In [ ]:
# plot the motif of interests from moda
motifs <- c('RORC', 'TBX21', 'IKZF2', 'STAT3',
            'STAT5A','STAT5B', 'BCL6', 'JUND', 'NFATC3',
           'NR3C1', 'NFAT5')
#motifs <-plotVarDev %>% as_tibble() %>% distinct(name) %>% pull(name)
markerMotifs <- getFeatures(cd4na_atac, select = paste(motifs, collapse="|"), useMatrix = "MotifMatrix")
markerMotifs <- grep("z:", markerMotifs, value = TRUE)
markerMotifs
# marker_genes <- rowData(cd4t_atac_gene_cores)$name %>% str_subset('GATA3|RORC|FOXP3|TBX21|IKZF2|STAT3')
p2 <-  plotEmbedding(
    ArchRProj = cd4na_atac, 
    colorBy = "ChromvarMatrix", 
    name = markerMotifs, 
    embedding = "UMAP",
    imputeWeights = getImputeWeights(cd4na_atac)
)
p2
p2 <-  plotEmbedding(
    ArchRProj = cd4na_atac, 
    colorBy = "GeneScoreMatrix", 
    name = motifs, 
    embedding = "UMAP",
    imputeWeights = getImputeWeights(cd4na_atac)
)
p2
p3 <-  plotEmbedding(
    ArchRProj = cd4na_atac, 
    colorBy = "GeneExpressionMatrix", 
    name = motifs, 
    embedding = "UMAP",
    imputeWeights =  getImputeWeights(cd4na_atac)
)
p3
plotPDF(p1, p2, p3, name = paste0(proj_name, '_tfs_umap.pdf'), ArchRProj = cd4na_atac,
        addDOC = FALSE, width = 4, height = 4)

In [ ]:
p2 <-  plotEmbedding(
    ArchRProj = cd4na_atac, 
    colorBy = "ChromvarMatrix", 
    name = markerMotifs, 
    embedding = "UMAP",
    imputeWeights = getImputeWeights(cd4na_atac)
)

In [ ]:
# marker_genes <- rowData(cd4t_atac_gene_cores)$name %>% str_subset('GATA3|RORC|FOXP3|TBX21|IKZF2|STAT3')
marker_genes <- c('IFNAR1')
marker_genes
p2 <-  plotEmbedding(
    ArchRProj = cd4na_atac, 
    colorBy = "GeneScoreMatrix", 
    name = marker_genes, 
    embedding = "UMAP",
    imputeWeights = getImputeWeights(cd4na_atac)
)
p2
p3 <-  plotEmbedding(
    ArchRProj = cd4na_atac, 
    colorBy = "GeneExpressionMatrix", 
    name = marker_genes, 
    embedding = "UMAP",
    imputeWeights = NULL
)
p3


In [ ]:
grid::grid.newpage()
grid::grid.draw(p$`IFNAR1`)

In [ ]:
grid::grid.newpage()
grid::grid.draw(p$`IL21`)

In [ ]:
grid::grid.newpage()
grid::grid.draw(p$IL6R)


In [ ]:
grid::grid.newpage()
grid::grid.draw(p$IL4R)

In [ ]:
grid::grid.newpage()
grid::grid.draw(p$IL13RA1)

## run dap analysis in cd4 naive

In [ ]:
cd4na_Matrix <- MOCHA::getCellPopMatrix(SampleTileMatricesAnnotated, "cd4_naive")
cd4na_Matrix[1:5, 1:5]

In [ ]:
# check the signal distribution in all the cell types
# extract all the tile matrix for all population
tile_dt <- lapply(cellPopulations, function(x){
    tile_mx <- getCellPopMatrix(SampleTileMatricesAnnotated, x) %>% as.data.table(keep.rownames = 'tiles') %>% 
        pivot_longer(cols = -tiles,names_to = 'Sample', 
                     values_to = 'Normalized_counts') %>% 
    mutate(cell_type=x) %>%
        as.data.table()
    meta_data <- colData(SampleTileMatricesAnnotated) %>% as_tibble() %>% 
    dplyr::select(subject_id, cohort, Sex, Sample)
    tile_mx <-tile_mx %>% left_join(meta_data, by='Sample')%>%
        as.data.table()
}) %>% rbindlist()

In [ ]:
tile_dt %>% head()

In [ ]:
p1 <- tile_dt %>% ggplot()+ 
    geom_histogram(aes(x=Normalized_counts, fill=cohort), 
                   position = 'identity', alpha=0.6)+
    facet_wrap(vars(cell_type))+
    scale_fill_manual(values=c("#69b3a2", "#404080")) 
p1
ggsave(file.path(fig_path, paste0(proj_name, 
                                  '_mocha_peaks_cell_type_histogram.pdf')), width = 12, height = 8)

In [ ]:
cd4na_Matrix[1:10,]
summary(cd4na_Matrix)

In [ ]:
cd4na_Matrix[1:5, 1:5]

In [ ]:
SampleTileMatricesAnnotated
rowData(SampleTileMatricesAnnotated) %>% as_tibble() %>% group_by(tileType) %>% tally()

In [ ]:
rowData(SampleTileMatricesAnnotated)$cd4_naive %>% unique()

In [ ]:
Log2Intensity <- SampleTileMatricesAnnotated@metadata$Log2Intensity
Log2Intensity

In [ ]:
cd4na_intra <- log2(getCellPopMatrix(SampleTileObj = SampleTileMatricesAnnotated[rowRanges(SampleTileMatricesAnnotated)$tileType == 
                                                 'Intragenic',], 
                                    cellPopulation = 'cd4_naive') + 1)
cd4na_prom <- log2(getCellPopMatrix(SampleTileObj = SampleTileMatricesAnnotated[rowRanges(SampleTileMatricesAnnotated)$tileType == 
                                                 'Promoter',], 
                                    cellPopulation = 'cd4_naive') + 1)
cd4na_dis <- log2(getCellPopMatrix(SampleTileObj = SampleTileMatricesAnnotated[rowRanges(SampleTileMatricesAnnotated)$tileType == 
                                                 'Distal',], 
                                    cellPopulation = 'cd4_naive') + 1)

In [ ]:
# cd4_values[1:5, 1:5]

In [ ]:
p1 <- hist(cd4na_intra)
p2 <- hist(cd4na_prom)
p3 <- hist(cd4na_dis)

In [ ]:
colData(SampleTileMatricesAnnotated)$subject_id

In [ ]:
# remove BR2024 sample and rerun the DAPs
SampleTileMatricesAnnotated_fl <- 
    SampleTileMatricesAnnotated[, colData(SampleTileMatricesAnnotated)$subject_id!='BR2024']
SampleTileMatricesAnnotated_fl

In [ ]:
####################################################
# 5. Get differential accessibility for specific 
#    cell populations. Here we are comparing MAIT  
#    cells between samples where our groupColumn 
#    "COVID_status" is Positive (our foreground) 
#    to Negative samples (our background).
####################################################
threshold <- 0.2
groupColumn <- "cohort"
join <- "union"
numCores <- 58

cd4na_diff <- MOCHA::getDifferentialAccessibleTiles(
    SampleTileObj = SampleTileMatricesAnnotated,
    cellPopulation = "cd4_naive",
    groupColumn = groupColumn,
    foreground = "pre-RA",
    background = "Healthy",
    fdrToDisplay = 0.2,
    signalThreshold = 14,
   # minZeroDiff = 0,
    numCores = numCores
)

In [ ]:
cd4na_diff <- annotateTiles(cd4na_diff, TxDb = TxDb, Org = Org)

In [ ]:
cd4na_diff %>% as_tibble() %>% filter(FDR<0.1 & P_value<0.05) %>% group_by(tileType) %>% tally()

In [ ]:
cd4na_diff$P_value %>% hist()
cd4na_diff$FDR %>% hist()

In [ ]:
cd4na_diff %>% as_tibble() %>% ggplot(aes(x=P_value, y=FDR)) + 
  geom_point(size=0.8) 

In [ ]:
cd4na_diff_table <- cd4na_diff %>% as_tibble() %>% filter(!is.na(P_value)) %>% 
    rstatix::adjust_pvalue(p.col = "P_value", output.col = 'p_adj', method = "BY")

In [ ]:
cd4na_diff_table %>% arrange(p_adj) %>% head()

In [ ]:
filtered_res <- cd4na_diff_table
  if (!all(is.na(filtered_res$P_value[filtered_res$P_value <= 0.95]))) {
    pi0_reduced <- qvalue::pi0est(filtered_res$P_value[filtered_res$P_value <= 0.95],
      pi0.method = "bootstrap",
      lambda = seq(0, 0.6, .05)
    )

    filtered_res$FDR_test <- qvalue::qvalue(filtered_res$P_value, pi0 = pi0_reduced$pi0)$qvalues
  } else {
    filtered_res$FDR_test <- NA # TODO Handle appropriately
  }

In [ ]:
# filtered_res %>% dplyr::select(P_value, FDR, FDR_test, p_adj) %>% filter(FDR<0.2)

In [ ]:
cd4na_diff <- cd4na_diff %>% annotateTiles(TxDb = TxDb,
                          Org = Org)

In [ ]:
cd4na_diff$tileType %>% table()

In [ ]:
cd4na_diff %>% as_tibble() %>% filter(FDR<0.2) %>% group_by(tileType) %>% tally()

In [ ]:
cd4na_diff %>% as_tibble() %>% head()

In [ ]:
cd4na_diff %>% saveRDS(file.path(output_path, 
                                  paste0(proj_name, '_MOCHA_total_cd4na_ravshealthy_DAPs.rds')))

In [ ]:
# load differential DAPS between pre-RA and healthy in cd4 naive
cd4na_diff <- readRDS(file.path(output_path, 
                                  paste0(proj_name, '_MOCHA_total_cd4na_ravshealthy_DAPs.rds')))

In [ ]:
cd4na_diff %>% as_tibble() %>% write_csv(file.path(output_path, paste0(proj_name, 'cd4na_atac_daps.csv')))

In [ ]:
cd4na_diff %>% as_tibble() %>% filter(FDR<0.1 & P_value<0.05) %>% group_by(tileType) %>% tally()

In [ ]:
cd4na_diff %>% as_tibble()%>%pull(P_value)%>%hist()

In [ ]:
cd4na_diff_sig = cd4na_diff %>% as_tibble() %>% filter(FDR<0.1 & P_value<0.05) 
cd4na_diff_sig%>%group_by(tileType)%>%tally()

In [ ]:
cd4na_diff_sig%>%dim()

In [ ]:
cd4na_diff %>% as_tibble() %>% filter(FDR<0.1 & P_value<0.05 & str_detect(Gene, 'IL21|IL6R'))

In [ ]:
# cd4na_diff %>% as_tibble() %>% filter(Gene=='IL21R')

In [ ]:
# take the significant peaks
# cd4na_diff_sig <- cd4na_diff[cd4na_diff$FDR<0.2&cd4na_diff$P_value<0.05]
# filter significant daps
cd4na_diff_sig <- cd4na_diff[(!is.na(elementMetadata(cd4na_diff)[,'FDR'])) & 
                                               (!is.na(elementMetadata(cd4na_diff)[,'Log2FC_C']))&
                             (elementMetadata(cd4na_diff)[,'FDR']<0.2)]

In [ ]:
cd4na_diff_sig

In [ ]:
SampleTileMatricesAnnotated[rowRanges(SampleTileMatricesAnnotated)$tileType == 'Intragenic',] %>% head()

In [ ]:
# plot the valcano plot of the peaks
cd4na_diff_table <- cd4na_diff %>% as_tibble() %>% 
    mutate(label_gene=if_else(FDR<0.2& P_value<0.05& abs(Log2FC_C) > 0.25, Gene, NA))

In [ ]:
cd4na_prom_gens = cd4na_diff_table%>%filter(FDR<0.2& P_value<0.05& abs(Log2FC_C) > 0.1 & tileType=='Promoter')%>%
    distinct(Gene)%>%pull(Gene)

In [ ]:
cd4na_prom_gens%>%length()

In [ ]:
# gene enrichment 
# promoter_up <- cd4na_prom_diff %>% as_tibble() %>% filter(P_value<0.05&FDR<0.2) %>% 
#     dplyr::filter(tileType=='Promoter') %>% pull(Gene) %>%
#     str_split(', ') %>% unlist()%>% unique() 
# promoter_up %>% length()
# pathway enrichment for cd4na
enrich_cd4na_promoter_up <- enrichr(cd4na_prom_gens, c('KEGG_2021_Human', 'MSigDB_Hallmark_2020'))


In [ ]:
# enrich_cd4na_promoter_up['KEGG_2021_Human']

In [ ]:
# plot the valcano plot of the peaks
cd4na_diff_table <- cd4na_diff %>% as_tibble() %>% 
    mutate(label_gene=if_else(FDR<0.2& abs(Log2FC_C) > 0.25, Gene, NA))
p1 <-  cd4na_diff_table%>%
    EnhancedVolcano::EnhancedVolcano(
    lab = cd4na_diff_table %>% pull(label_gene),
    x = 'Log2FC_C',
    pCutoff = 0.2,
    FCcutoff = 0.25,
    y = 'FDR', 
    title = 'CD4naive pre-RA vs healthy', subtitle = NULL,
    xlim = c(-0.5, 0.5),
    ylim = c(0.4, 1)) 
p1
ggsave(file.path(fig_path, paste0(proj_name, '_mocha_DAPs_valcano_plot.png')),
       width = 8, height = 8)

In [ ]:
fig_path

In [ ]:
cd4na_diff_table %>% filter(str_detect(Gene,'NFAT|STAT|JUN|FOS|BATF|BCL6|GATA')&
                            FDR<0.2&P_value<0.1)

### run DAPs only in promoters

In [ ]:
#taget genes
target_genes<-rowData(SampleTileMatricesAnnotated)$Gene%>%unique()%>%
    str_subset('IL\\d|STAT|NFAT|CXCR5|MAF|CXCL|TGFB|TNF|IKZF|ICOS|TIGIT|BCL6')%>%
    str_subset('MICOS10|C1Q|MAFA|PIWIL2', negate = TRUE)
target_genes%>%length()
target_genes

In [ ]:
colData(SampleTileMatricesAnnotated)$subject_id%>%unique()

In [ ]:
# # remove BR2024 sample and rerun the DAPs
# SampleTileMatricesAnnotated_fl <- 
#     SampleTileMatricesAnnotated[, colData(SampleTileMatricesAnnotated)$subject_id!='BR2024']

# fileter 
SampleTileMatricesAnnotated_prom <- SampleTileMatricesAnnotated[!is.na(rowData(
    SampleTileMatricesAnnotated)$Gene) & 
    rowData(SampleTileMatricesAnnotated)$Gene%in%target_genes,]
SampleTileMatricesAnnotated_prom

In [ ]:
# rowData(SampleTileMatricesAnnotated)$Gene%>%str_subset('IL\\d|STAT|NFAT|CXCR5')

In [ ]:
####################################################
# 5. Get differential accessibility for specific 
#    cell populations. Here we are comparing MAIT  
#    cells between samples where our groupColumn 
#    "COVID_status" is Positive (our foreground) 
#    to Negative samples (our background).
####################################################
threshold <- 0.2
groupColumn <- "cohort"
join <- "union"
numCores <- 58

cd4na_prom_diff <- MOCHA::getDifferentialAccessibleTiles(
    SampleTileObj = SampleTileMatricesAnnotated_prom,
    cellPopulation = "cd4_naive",
    groupColumn = groupColumn,
    foreground = "pre-RA",
    background = "Healthy",
    fdrToDisplay = 0.2,
    signalThreshold = 15,
    #minZeroDiff = 0,
    numCores = numCores
)

In [ ]:
cd4na_prom_diff$FDR %>% hist()
cd4na_prom_diff$P_value %>% hist()

In [ ]:
cd4na_prom_diff <- cd4na_prom_diff %>% annotateTiles(TxDb = TxDb, Org = Org)

In [ ]:
cd4na_prom_diff %>% as_tibble() %>% 
    filter(FDR<0.2&P_value<0.05) %>% 
    group_by(tileType) %>% tally()

In [ ]:
cd4na_prom_diff

In [ ]:
# check promoters that have significant changes
cd4na_dap_table <- cd4na_prom_diff %>%as_tibble()
cd4na_dap_table %>% group_by(tileType) %>% tally()
cd4na_dap_table %>% head()

In [ ]:
cd4na_dap_table%>%filter(P_value<0.05&FDR<0.2&str_detect(Gene, 'IL21|CXCR|MAF|IL17|IL23|TNF|IKZF|IL\\d'))

In [ ]:
cd4na_dap_table%>%filter(P_value<0.05&str_detect(Gene, 'CXCL'))

In [ ]:
cd4na_prom_diff %>% as_tibble() %>% filter(FDR<0.2&P_value<0.05, str_detect(Gene, 'IL\\d|NFAT|STAT|FOS|JUN'))

In [ ]:
cd4na_prom_diff %>% as_tibble() %>% head()

In [ ]:
# gene enrichment 
promoter_up <- cd4na_prom_diff %>% as_tibble() %>% filter(P_value<0.05&FDR<0.2) %>% 
    dplyr::filter(tileType=='Promoter') %>% pull(Gene) %>%
    str_split(', ') %>% unlist()%>% unique() 
promoter_up %>% length()
# pathway enrichment for cd4na
enrich_cd4na_promoter_up <- enrichr(promoter_up, c('KEGG_2021_Human', 'MSigDB_Hallmark_2020'))


In [ ]:
enrich_cd4na_promoter_up$MSigDB_Hallmark_2020 %>% 
    dplyr::select(-c(Old.P.value,Old.Adjusted.P.value)) %>% filter(Adjusted.P.value<0.05)

In [ ]:
# plot the valcano plot of the peaks
cd4na_prom_diff_table <- cd4na_prom_diff %>% as_tibble() %>% 
    mutate(label_gene=if_else(FDR<0.3& abs(Log2FC_C) > 0.1, Gene, NULL))
p1 <-  cd4na_prom_diff_table%>%
    EnhancedVolcano::EnhancedVolcano(
    lab = cd4na_prom_diff_table %>% pull(label_gene),
    x = 'Log2FC_C',
    pCutoff = 0.3,
    FCcutoff = 0.25,
    y = 'FDR', 
    title = 'CD4naive pre-RA vs healthy', subtitle = NULL,
    xlim = c(-0.5, 0.5),
    ylim = c(0.4, 1)) 
p1
ggsave(file.path(fig_path, paste0(proj_name, '_promoter_mocha_DAPs_valcano_plot.png')),
       width = 8, height = 8)

In [ ]:
# cd4na_prom_diff_table %>% arrange(P_value, desc(Log2FC_C)) %>%
#     filter(!is.na(Gene)) %>% filter(str_detect(Gene, ''))

### load ALTRA AIM1 DAP results for CD4 naive

In [ ]:
aim1_cd4na_daps<- readRDS('/home/jupyter/data/ra_longitudinal/ATAC/ALTRA_Aim1_DAP_CD4Naive_granges.rds')

In [ ]:
aim1_cd4na_daps <- aim1_cd4na_daps%>% annotateTiles(TxDb = TxDb,
                          Org = Org)

In [ ]:
aim1_cd4na_daps

In [ ]:
aim1_cd4na_daps%>%filter(P_value<0.05)%>%as_tibble()%>%pull(Gene)%>%unique()

In [ ]:
aim1_cd4na_daps%>%filter(Gene%in%c( 'IL21', 'CXCR5', 'MAF' , 'IL23', 'CXCL13')) 

### Run dap analysis in CD4 naive cells only in enhancer regions

In [ ]:
# load the enhancer and promoter reference 
prom_enh_hg38 <- read_tsv('/home/jupyter/data/Reference/ActivePromoterEnhancerLinks_hg38.tsv',
                         show_col_types = FALSE)
prom_enh_hg38 %>% head()
# extract the links for cd4 naive cells
prom_enh_cd4na <- prom_enh_hg38 %>% filter(str_detect(celltypes, 'nCD4'))

In [ ]:
# prom_enh_cd4na %>% janitor::get_dupes(oeID) #%>% distinct(oeID)

In [ ]:
# get the enhancer region for cd4na
prom_enh_cd4na_enh <- prom_enh_cd4na %>%
    makeGRangesFromDataFrame(seqnames.field='seqnames_oe',
                                            start.field="start_oe",
                                            end.field='end_oe',
                                           keep.extra.columns=TRUE) 
prom_enh_cd4na_enh %>% head()

In [ ]:
# subset the peaks set to known promoter and enhancer regions
SampleTileMatricesAnnotatedfl <- subsetByOverlaps(SampleTileMatricesAnnotated, prom_enh_cd4na_enh) 
SampleTileMatricesAnnotatedfl

In [ ]:
SampleTileMatricesAnnotatedfl

In [ ]:
####################################################
# 5. Get differential accessibility for specific 
#    cell populations. Here we are comparing MAIT  
#    cells between samples where our groupColumn 
#    "COVID_status" is Positive (our foreground) 
#    to Negative samples (our background).
####################################################
threshold <- 0.2
groupColumn <- "cohort"
join <- "union"
numCores <- 55

cd4na_diff_prom_enhancer <- MOCHA::getDifferentialAccessibleTiles(
    SampleTileObj = SampleTileMatricesAnnotatedfl,
    cellPopulation = "cd4_naive",
    groupColumn = groupColumn,
    foreground = "pre-RA",
    background = "Healthy",
    fdrToDisplay = 0.2,
    signalThreshold = 12,
    minZeroDiff = 0.5,
    numCores = numCores
)

In [ ]:
cd4na_diff_prom_enhancer$FDR %>% hist()

In [ ]:
# filter significant daps
cd4na_enhancer_sig <- cd4na_diff_prom_enhancer[(!is.na(elementMetadata(cd4na_diff_prom_enhancer)[,'FDR'])) & 
                                               (!is.na(elementMetadata(cd4na_diff_prom_enhancer)[,'Log2FC_C']))&
                             (elementMetadata(cd4na_diff_prom_enhancer)[,'FDR']<0.2)]

In [ ]:
cd4na_enhancer_sig

In [ ]:
# check what gene enhancers contains DAPs
enhancer_genes_up <- subsetByOverlaps(prom_enh_cd4na_enh, cd4na_enhancer_sig, minoverlap = 499)$Gene %>% 
    str_split(', ', simplify = TRUE) %>% c() %>%
    unique() 
enhancer_genes_up %>% length()

In [ ]:
subsetByOverlaps(prom_enh_cd4na_enh, cd4na_enhancer_sig, minoverlap = 499)%>%
    filter(str_detect(Gene, 'IL\\d'))

In [ ]:
enhancer_genes_up%>%str_subset('IL')

In [ ]:
# gene enrichment 
# pathway enrichment for cd4na
enrich_cd4na_enhancer_genes_up <- enrichr(enhancer_genes_up, c('KEGG_2021_Human', 'MSigDB_Hallmark_2020'))


In [ ]:
enrich_cd4na_enhancer_genes_up$MSigDB_Hallmark_2020 %>% 
    dplyr::select(-c(Old.P.value,Old.Adjusted.P.value))

In [ ]:
il6r_granges <- subsetByOverlaps(prom_enh_cd4na_enh, cd4na_diff_sig) %>% filter(str_detect(Gene,'IL6|IL4R|IL13R|IL\\d'))
il6r_granges

In [ ]:
# check the DAPs FDR<0.2 and also in the enha

In [ ]:
subsetByOverlaps(cd4na_diff_sig, il6r_granges)

In [ ]:
cd4na_diff %>% as_tibble() %>% 
    ggplot(aes(x=P_value)) +
    geom_histogram()
ggsave(file.path(fig_path, paste0(proj_name, '_mocha_DAPs_p_values_distribution.pdf')), width = 6, height = 6)

In [ ]:
cd4na_diff %>% as_tibble() %>% head()

In [ ]:
cd4na_dap_counts <- cd4na_diff_sig %>% as_tibble() %>% mutate(direction=if_else(Log2FC_C>0, 'up', 'down')) %>% 
    group_by(tileType, direction) %>% tally()
cd4na_dap_counts

In [ ]:
# subset the sample peak matrix by significant differential peaks
SampleTileMatricesAnnotated_diff <- subsetByOverlaps(SampleTileMatricesAnnotated,
                                                   cd4na_diff[(!is.na(elementMetadata(cd4na_diff)[,'FDR'])) &
                                                              (elementMetadata(cd4na_diff)[,'FDR']<0.2)])

In [ ]:
SampleTileMatricesAnnotated_diff

In [ ]:
# subset the sample tile matrix by promoter region
STM_diff_prom <- SampleTileMatricesAnnotated_diff[rowData(SampleTileMatricesAnnotated_diff)$tileType=='Promoter', ]

### run differential peaks in the genome region around the degs
limit the range od search for differential peaks in around 2m bps of the degs.
- step 1: load degs in overlap in the teaseq and validation dataset

### check the relationship between promoter DAPs and gene expression

In [ ]:
# load the annotate seurat dataset
cd4na_pseudo <- readRDS(file.path(data_path, 'PreRA_teaseq_cd4na_t_rna_subject_PseudoBulk.rds'))
cd4na_pseudo

In [ ]:
# load the degs for cd4 naive
# load the deg list
cd4na_pseudo_degs <- 
    read_csv('/home/jupyter/data/preRA_teaseq/output_results/cd4na_t/PreRA_teaseq_cd4na_tPseudoBulk_deseq2_degs_preRA.csv',
            show_col_types = FALSE) 
cd4na_pseudo_degs %>% head()

In [ ]:
cd4na_pseudo_degs$gene %>% length()

In [ ]:
# subset the cd4 naive
cd4na_pseudo <- ra_tea_l2_pseudo[cd4na_pseudo_degs$gene, 
                                 colData(ra_tea_l2_pseudo)$clean_l2_cell_types=='cd4_naive']
# make colnames of the pseudo object as subject id
colnames(cd4na_pseudo) <- colData(cd4na_pseudo)$subject_id
cd4na_pseudo <- scuttle::logNormCounts(cd4na_pseudo)

In [ ]:
logcounts(cd4na_pseudo) %>% head()

In [ ]:
# check what genes are also present in the degs list
cd4na_sig_degs <- cd4na_pseudo_degs %>% filter(padj<0.2) 
# table(rowData(STM_diff_prom)$Gene %in% cd4na_sig_degs$gene)

In [ ]:
# subset the genes which are differential peaks at promoter regions and also are degs
STM_diff_prom_gex <- STM_diff_prom[rowData(STM_diff_prom)$Gene %in% cd4na_sig_degs$gene]

In [ ]:
cd4na_sig_degs %>% filter(gene %in% rowData(STM_diff_prom)$Gene)

In [ ]:
TxDb <- AnnotationDbi::loadDb(S4Vectors::metadata(SampleTileMatricesAnnotated)$TxDb)
Org <- AnnotationDbi::loadDb(S4Vectors::metadata(SampleTileMatricesAnnotated)$Org)

In [ ]:
cd4na_diff_sig <- annotateTiles(cd4na_diff_sig, TxDb = TxDb, Org = Org)

In [ ]:
cd4na_diff_sig %>% as_tibble() %>% filter(tileType=='Promoter' & P_value<0.05 & (!Gene %in% cd4na_sig_degs$gene) & Log2FC_C>0.5) %>% distinct(Gene)

In [ ]:
# merge the ATAC promotor with gene expression
cd4na_diff_sig_gex <- cd4na_diff_sig %>% filter(tileType=='Promoter' & Gene %in% cd4na_sig_degs$gene)  %>% 
    as_tibble() %>% dplyr::rename('gene'='Gene') %>% 
    left_join(cd4na_sig_degs, by='gene', suffix = c('_dap', '_deg')) %>% 
    dplyr::rename('Log2FC_dap'='Log2FC_C', 'logFC_deg'='logFC', 'direction_deg'='direction') %>%
    mutate(direction_dap=if_else(Log2FC_dap>0, 'up', 'down'))
cd4na_diff_sig_gex %>% colnames()

In [ ]:
# plot the gene expression and DAP at promoters
p1 <- cd4na_diff_sig_gex %>% ggplot(aes(x=Log2FC_dap, y=logFC_deg, color=FDR_deg)) +
    geom_point() + 
    ggrepel::geom_text_repel(aes(label = gene)) + 
    geom_hline(yintercept=0) + geom_vline(xintercept=0)+
    theme_bw()+
    viridis::scale_color_viridis()
# p2 <-  cd4na_diff_sig_gex %>% ggplot(aes(x=Log2FC_dap, y=logFC_deg, color=FDR.dap)) +
#     geom_point() + ggrepel::geom_text_repel(aes(label = gene)) + viridis::scale_color_viridis()
# cowplot::plot_grid(p1, p2, s)
p1
ggsave(file.path(fig_path, paste0(proj_name, '_DEGs_DAPs_promoters.pdf')), width = 12, height = 8)

In [ ]:
cd4na_diff_sig_gex %>% filter(FDR_deg<0.05& direction_deg==direction_dap)

In [ ]:
cd4na_diff_sig_gex %>% filter(gene=='PLCG2')

In [ ]:
cd4na_diff_sig_gex %>% filter()

In [ ]:
# check what are peaks distribution for the significant differential peaks vs not
peak_dist <- rowData(SampleTileMatricesAnnotated) %>% as_tibble(rownames = 'ranges') %>% 
    mutate(sig_differential=ranges %in% rownames(SampleTileMatricesAnnotated_fl))
peak_dist %>% head()

In [ ]:
peak_dist %>% group_by(tileType, sig_differential) %>% tally()

In [ ]:
# plot the distribution of the differential peaks and all peaks called in the cd4naive
peak_dist_counts <- peak_dist %>% group_by(tileType, sig_differential) %>% tally()
p1 <- peak_dist_counts %>% ggplot(aes(x=sig_differential, y=n, fill=tileType, label = n)) +
    geom_bar(stat = "identity") +
    geom_text(position = position_stack(vjust = 0.5))+
    scale_y_continuous(trans='log10') + 
    scale_fill_manual(values = nejm_color)

p2 <- peak_dist %>% ggplot() +
    geom_bar(aes(x=sig_differential, fill=tileType), position = 'fill') +
    ylab('precentage')+ 
    scale_fill_manual(values = nejm_color)
p1+p2
ggsave(file.path(fig_path, paste0(proj_name, '_mocha_DAPs_distribusion.pdf')), width = 8, height = 4)

In [ ]:
# load the tf-enhancer data
prom_enhancer_ref <- read_tsv('/home/jupyter/data/Reference/ActivePromoterEnhancerLinks.tsv', show_col_types = FALSE)
prom_enhancer_ref %>% head()

In [ ]:
prom_enhancer_ref %>% filter(str_detect(`cellType(s)`, 'nCD4')) %>% head()

In [ ]:
prom_granges <- makeGRangesFromDataFrame(prom_enhancer_ref,
                         keep.extra.columns=TRUE,
                         ignore.strand=TRUE,
                         seqinfo=NULL,
                         seqnames.field=c("baitChr"),
                         start.field="baitSt",
                         end.field=c("baitEnd"),
                         starts.in.df.are.0based=FALSE)


### plot region

In [ ]:
# # load the sample tile matrix
SampleTileMatricesAnnotated <- readRDS(file.path(output_path,
                                        paste0(proj_name, '_MOCHA_total_cd4na_SampleTileMatrices.rds')))

In [ ]:
SampleTileMatricesAnnotated

In [ ]:
# make status column
colData(SampleTileMatricesAnnotated)$status = factor(if_else(colData(SampleTileMatricesAnnotated)$cohort=='pre-RA',
                                                      'ARI', 'CON2'), levels=c('CON2', 'ARI'))
SampleTileMatricesAnnotated$status%>%unique()

In [ ]:
# get the gene and and promoter which share the same direction
# cd4na_prom_gex_sig <- cd4na_diff_sig_gex %>% filter( direction_deg==direction_dap)
# cd4na_prom_gex_sig %>% arrange(FDR_deg) %>% distinct(gene, .keep_all = TRUE) 

In [ ]:
txList <- suppressWarnings(GenomicFeatures::transcriptsBy(TxDb, by = ("gene")))
names(txList) <- suppressWarnings(AnnotationDbi::mapIds(Org, names(txList), "SYMBOL", "ENTREZID"))

In [ ]:
names(txList) %>% str_subset('LCK')

In [ ]:
# rowData(SampleTileMatricesAnnotated)$Gene=='PLCG2'

In [ ]:
# cd4na_diff %>% #filter(FDR<0.2&P_value<0.1) %>% 
#     arrange(P_value) %>% 
#     as_tibble() %>% filter(str_detect(Gene, 'LCK'))

In [ ]:
# get the gene names and gene ranges
gene_name <- 'IL21'
# peak_ranges_str <- cd4na_prom_gex_sig %>% filter(gene==gene_name) 
gene_ranges <- txList[[gene_name]] %>% as_tibble() %>% 
    mutate(gene_range=paste0(seqnames, ':', start-2000, '-', end+2000))
gene_ranges
gene_ranges[1,]$gene_range

In [ ]:
####################################################
# 5. (Optional) Plot a specific region's coverage. 
#    Here we plot coverage at a specific region and 
#    gene by infection stage.
####################################################
countSE <- MOCHA::extractRegion(
  SampleTileObj = SampleTileMatricesAnnotated, 
  cellPopulations = 'cd4_naive',
  region = gene_ranges[1,]$gene_range, 
  groupColumn = 'status',
 # approxLimit = 120000,
    numCores = 15,
  sampleSpecific = FALSE
)

In [ ]:
# p1 <- MOCHA::plotRegion(countSE = countSE, motifSetName='Motifs',
#                         motif_lab_size=4, motif_lab_alpha=0.5, motif_line_alpha=0.5, 
#                         whichGene=gene_name)
# p1

In [ ]:
# add delta
countSE <- addAccessibilityShift(countSE,
    foreground = "cd4_naive.ARI",
    background = "cd4_naive.CON2")

In [ ]:
pdf(file.path(fig_path, paste0(proj_name, '_peak_regions_preRAvsHealthy_',
                               gene_name,'_status.pdf')), 
    width = 4, height = 4)
# Note that to show specific genes with the option 
# 'whichGene' you must have the package RMariaDB
# installed
p1 <- MOCHA::plotRegion(countSE = countSE, #showIdeogram=FALSE,
                        whichGene=gene_name, counts_group_colors=c('#F59F00', '#5AAA46', '#840032'))
p1
dev.off()

In [ ]:
p1 <- MOCHA::plotRegion(countSE = countSE, motifSetName='Motifs_fl',
                        motif_lab_size=4, motif_lab_alpha=0.5, motif_line_alpha=0.5, 
                        whichGene=gene_name, counts_group_colors=c('#F59F00', '#5AAA46', '#840032'))
p1

In [ ]:
names(metadata(countSE)$Motifs)%>%str_subset('BCL6|MAF|ROR|SMAD|STAT|BATF|IRF4|TBX21')

In [ ]:
# metadata(countSE)$Motifs%>%names()
motif_plot = c('BCL6', 'STAT3')
metadata(countSE)$Motifs_Trim = metadata(countSE)$Motifs[names(metadata(countSE)$Motifs)%in% motif_plot]

In [ ]:
IL21_peaks <- GRanges(seqnames = "chr4", ranges = IRanges(start = 122617000, end = 122617999))

print('BCL6')
metadata(countSE)$Motifs_Trim$BCL6%>%subsetByOverlaps(IL21_peaks,  type="any")


In [ ]:
print('STAT3')
metadata(countSE)$Motifs_Trim$STAT3%>%subsetByOverlaps(IL21_peaks,  type="any")

In [ ]:
fig_path

In [ ]:
pdf(file.path(fig_path, paste0(proj_name, '_peak_regions_preRAvsHealthy_',
                               gene_name,'_status_motif.pdf')), 
    width = 10, height = 10)
p1 <- MOCHA::plotRegion(countSE = countSE, motifSetName='Motifs_Trim',
                        motif_lab_size=3, motif_lab_alpha=0.5, motif_line_alpha=0.5,  
                        whichGene=gene_name, counts_group_colors=c('#F59F00', '#5AAA46', '#840032'))
p1
dev.off()

In [ ]:
# define a function to plot gene tracks
PlotPromoterRegion <- function(SampleTileObj, gene_name, cellPopulations, groupColumn, numCores=15,
                              sampleSpecific = FALSE, ...) {
    gene_ranges <- txList[[gene_name]] %>% as_tibble() %>% 
        mutate(gene_range=paste0(seqnames, ':', start-2000, '-', end+2000))
    countSE <- MOCHA::extractRegion(
      SampleTileObj = SampleTileObj, 
      cellPopulations = cellPopulations,
      region = gene_ranges$gene_range[1], 
      groupColumn = groupColumn,
      numCores = numCores,
      sampleSpecific = sampleSpecific
    )
        # add delta
    # add delta
    countSE <- addAccessibilityShift(countSE,
        foreground = paste0(cellPopulations, ".ARI"),
        background = paste0(cellPopulations, ".CON2"))
    pdf(file.path(fig_path, paste0(proj_name,'atac_annotate_',cellPopulations, '_',
                                   gene_name,'_gene_regions.pdf')), width = 5, height = 5)
    p1 <- MOCHA::plotRegion(countSE = countSE, whichGene = gene_name, ...)
    print(p1)
    dev.off()   
    return(p1)
}

In [ ]:
SampleTileMatricesAnnotated

In [ ]:
gene_name = 'IL6R'
txList[[gene_name]] %>% as_tibble() %>% 
        mutate(gene_range=paste0(seqnames, ':', start-2000, '-', end+2000))

In [ ]:
chr1:154457000-154457499	

In [ ]:
PlotPromoterRegion(SampleTileObj = SampleTileMatricesAnnotated, gene_name = 'IL6R',
                   cellPopulations='cd4_naive',groupColumn='status', 
                   counts_group_colors=c('#F59F00', '#5AAA46', '#840032'))

In [ ]:
countSE <- MOCHA::extractRegion(
  SampleTileObj = SampleTileMatricesAnnotated, 
  cellPopulations = 'cd4_naive',
  region = gene_ranges[1,]$gene_range, 
  groupColumn = 'status',
 # approxLimit = 120000,
    numCores = 15,
  sampleSpecific = FALSE
)
# add delta
countSE <- addAccessibilityShift(countSE,
    foreground = "cd4_naive.ARI",
    background = "cd4_naive.CON2")


In [ ]:
# export to bigwig
exportCoverage(
  SampleTileMatricesAnnotated,
  dir = '/home/jupyter/data/preRA_teaseq/output_results/atac',
  type = TRUE,
  cellPopulations = "cd4_naive",
  groupColumn = 'status',
  sampleSpecific = FALSE,
  saveFile = TRUE,
  numCores = 15,
  verbose = FALSE
)

In [ ]:
filter=(rowData(SampleTileMatricesAnnotated)$Gene %in% gene_names&
        rowData(SampleTileMatricesAnnotated)$tileType == 'Promoter')
filter[is.na(filter)]=FALSE

prom_tiles <- assays(SampleTileMatricesAnnotated[filter, ])$cd4_naive %>% as_tibble(rownames = 'tiles') %>% 
    pivot_longer(cols = -tiles, names_to = 'Sample', values_to = 'normalized_counts')%>% 
    left_join(rowData(SampleTileMatricesAnnotated[filter, ])%>% as_tibble(rownames = 'tiles'), by='tiles')

In [ ]:
assays(SampleTileMatricesAnnotated[filter, ])$cd4_naive %>% as_tibble(rownames = 'tiles')

In [ ]:
SampleTileMatricesAnnotated[filter, ]%>% colData() %>% as_tibble() %>% select(Sample, subject_id)
SampleTileMatricesAnnotated[filter, ] %>% rowData()

In [ ]:
# plot the corresponding gene 
gene_names <-c('FOS', 'JUND', 'EGR1', 'IRF7', 'IL17RA')
gex_expression <- assays(cd4na_pseudo[gene_names, ])$logcounts %>% as_tibble(rownames = 'gene') %>% 
    pivot_longer(cols = -gene, names_to = 'subject_id', values_to = 'expression') %>%
    left_join(cd4na_pseudo %>% colData() %>% as_tibble() %>% select(subject_id, cohort, age), by='subject_id')
gex_expression %>% head()

In [ ]:
gex_expression %>% ggplot(aes(x=cohort, y=expression))+
    geom_violin(aes(fill=cohort))+
    geom_jitter() + facet_wrap(vars(gene), scales = 'free_y')

## examine peaks associated with the Factor1 in mofa

In [ ]:
# load mofa factor 1 values 
factor_values <- read_tsv('/home/jupyter/data/preRA_teaseq/output_results/MOFA/Mofa_preRA_cd4na_07192023_factor_values.tsv')%>%
    mutate(subject_id=sample)

f1_values <- factor_values %>% filter(factor=='Factor1') %>%
    mutate(subject_id=sample, f1_value=value)
f1_values

In [ ]:
# make factor values as a summrized experiment
factor_assay <- factor_values %>% dplyr::select(subject_id,factor,value) %>%
    pivot_wider(id_cols = 'factor', names_from = 'subject_id', values_from = 'value') %>%
    as.data.frame()
rownames(factor_assay) = factor_assay$factor
factor_assay <- factor_assay[, -1] %>% as.matrix()
factor_coldata <- factor_values %>% dplyr::select(subject_id, sample.sampleKitGuid:cohort) %>%
    distinct(sample.sampleKitGuid, .keep_all = TRUE)%>%
    as.data.frame() 
rownames(factor_coldata) = factor_coldata$subject_id
# 
factor_se <- SummarizedExperiment(assays = list(factors=factor_assay),
                                 colData=factor_coldata)
factor_se

In [ ]:
# add factor 1
# remove BR2024 sample from the atac data
SampleTileMatricesAnnotated_fl <- 
    SampleTileMatricesAnnotated[, colData(SampleTileMatricesAnnotated)$subject_id!='BR2024']

meta_data <- colData(SampleTileMatricesAnnotated_fl) %>% as_tibble() %>% 
    left_join(dplyr::select(f1_values, c(subject_id, f1_value)), by='subject_id')
colData(SampleTileMatricesAnnotated_fl)$f1_value=meta_data$f1_value

In [ ]:
colData(SampleTileMatricesAnnotated_fl) %>% colnames()

In [ ]:
# # change the sample sample names to subject_id
# colnames(SampleTileMatricesAnnotated_fl) <- colData(SampleTileMatricesAnnotated_fl)$subject_id
# SampleTileMatricesAnnotated_fl

In [ ]:
counts(SampleTileMatricesAnnotated_fl) = assay(SampleTileMatricesAnnotated_fl, 'cd4_naive')

In [ ]:
SampleTileMatricesAnnotated_fl

In [ ]:
# source('/home/jupyter/github/ChAI/R/association_functions.R')
# source('/home/jupyter/github/ChAI/R/model_functions.R')

In [ ]:
library('ChAI')

In [ ]:
# test_atac <- ChAI::scATAC_Associations(SampleTileMatricesAnnotated_fl, 
#                                  sampleColumn='subject_id',
#                                  numCores = 1, cellPopulation = 'counts',
#                     continuousFormula='exp1 ~ exp2', 
#                                  generalList=c('Factor1', 'Factor2', 'Factor3', 'Factor4', 'Factor5'),
#                     ziFormula='exp1 ~ 0 + exp2', 
#                                  tileList=cd4na_diff_sig,
#                     generalSE = factor_se, generalAssay='factors')

In [ ]:
# add f1 values into metadata
metadata(SampleTileMatricesAnnotated_fl)$f1_value = dplyr::select(as.data.frame(colData(SampleTileMatricesAnnotated_fl)), f1_value)
metadata(SampleTileMatricesAnnotated_fl)$age = dplyr::select(as.data.frame(colData(SampleTileMatricesAnnotated_fl)), age)

In [ ]:
MetaDF <- as.data.frame(SummarizedExperiment::colData(SampleTileMatricesAnnotated_fl))
MetaDF %>% colnames()

In [ ]:
metadata(SampleTileMatricesAnnotated_fl)$FragmentCounts
SampleTileMatricesAnnotated

In [ ]:
# find out the sample to be remove
sampleid_2024 <- colData(SampleTileMatricesAnnotated) %>% as_tibble() %>% dplyr::select(Sample, subject_id) %>% 
    filter(subject_id=='BR2024') %>%pull(Sample)
sampleid_2024
metadata(SampleTileMatricesAnnotated_fl)$FragmentCounts = 
    metadata(SampleTileMatricesAnnotated_fl)$FragmentCounts[-sampleid_2024,]

In [ ]:
metadata(SampleTileMatricesAnnotated_fl)$FragmentCounts[-sampleid_2024,]

In [ ]:
metadata(SampleTileMatricesAnnotated_fl)$FragmentCounts[
    rownames(metadata(SampleTileMatricesAnnotated_fl)$FragmentCounts)!=sampleid_2024,]

In [ ]:
colData(SampleTileMatricesAnnotated_fl)$FragmentCounts

In [ ]:
colData(SampleTileMatricesAnnotated_fl)$FragNumber = metadata(SampleTileMatricesAnnotated_fl)$FragmentCounts[
    rownames(metadata(SampleTileMatricesAnnotated_fl)$FragmentCounts)!=sampleid_2024,]

In [ ]:
SampleTileMatricesAnnotated_fl

In [ ]:
# run zero inflated association for the f1 values and atac peaks 
f1_peaks <- ChAI::model_scATAC(atacSE = SampleTileMatricesAnnotated_fl, numCores = 60,
                                ziFormula = '~ FragNumber + f1_value + age',
                                continuousFormula='exp ~ f1_value + age', 
                                    cellPopulation = 'counts', verbose = TRUE)

In [ ]:
f1_peaks

In [ ]:
f1_peaks %>% saveRDS(file.path(output_path, paste0(proj_name, '_peaks_mofa_factor1_association.rds')))

In [ ]:
f1_peaks <- readRDS(file.path(output_path, paste0(proj_name, '_peaks_mofa_factor1_association.rds')))

In [ ]:
file.path(output_path, paste0(proj_name, '_peaks_mofa_factor1_association.rds'))

In [ ]:
# get the promoters that significant associated with factor1
f1_sig_promoter <- f1_peaks[((!is.na(assay(f1_peaks, 'f1_value')$FDR)&
         assay(f1_peaks, 'f1_value')$FDR<0.05)|
        (!is.na(assay(f1_peaks, 'ZI_f1_value')$FDR)& assay(f1_peaks, 'ZI_f1_value')$FDR<0.05)) & 
     (rowData(f1_peaks)$tileType =='Promoter'), ]
f1_sig <- f1_peaks[((!is.na(assay(f1_peaks, 'f1_value')$FDR)&
         assay(f1_peaks, 'f1_value')$FDR<0.01)), ]

In [ ]:
assay(f1_peaks, 'ZI_f1_value')$FDR %>% hist()
assay(f1_peaks, 'f1_value')$FDR %>% hist()

In [ ]:
f1_sig
rowData(f1_sig) %>% as_tibble() %>% group_by(tileType) %>% tally()

In [ ]:
rowData(f1_sig)

In [ ]:
# get the significant genes associated with Facoter 1
f1_sig_promoter <- rowData(f1_sig)[rowData(f1_sig)$tileType=='Promoter',]
promoter_genes = f1_sig_promoter%>%as_tibble()%>%pull(Gene)%>%unique()
# promoter_genes %>% length()
promoter_genes %>% str_subset('IL\\d|CD\\d')

In [ ]:
f1_sig_promoter_pw <- enrichr(promoter_genes, c('KEGG_2021_Human', 'MSigDB_Hallmark_2020'))


In [ ]:
f1_sig_promoter_pw['MSigDB_Hallmark_2020']

In [ ]:
# load the factor weight
f1_weight <- read_tsv('/home/jupyter/data/preRA_teaseq/output_results/MOFA/Mofa_preRA_cd4na_07192023_factor_weights.tsv') %>%
    filter(factor=='Factor1'&view=='rna') %>% mutate(Gene=str_remove(feature, 'rna_'))
f1_weight %>% head()

In [ ]:
# get mofit annotation
f1_motifs <- chromVAR::getAnnotations(metadata(SampleTileMatricesAnnotated_fl)$Motifs, 
                                      rowRanges = rowRanges(f1_sig_promoter))
rowData(f1_motifs)$f1_value_FDR=assay(f1_sig_promoter, 'f1_value')$FDR
rowData(f1_motifs)$f1_value_Estimate=assay(f1_sig_promoter, 'f1_value')$Estimate

In [ ]:
f1_motifs

In [ ]:
# f1_motifs
# filter genes that associated with NFAT family
nfat = colnames(assay(f1_motifs, 'annotationMatches')) %>% str_subset('NFAT|STAT3')
nfat
# assay(f1_motifs, 'annotationMatches')[, nfat][rowSums(assay(f1_motifs, 'annotationMatches')[, nfat])>=1,]

In [ ]:
# filter out the granges by NFAT
nfat_motif <- f1_motifs[rowSums(assay(f1_motifs, 'annotationMatches')[, nfat])>=1, ]
rowData(nfat_motif)$tileType %>% table()
nfat_motif


In [ ]:
# join the NFAC genes associated with 
NFAT_genes <- rowData(nfat_motif) %>% 
    as_tibble(rownames = 'peak') %>% left_join(f1_weight, by='Gene')
NFAT_genes %>% filter(abs(value)>0.6) %>% head()

In [ ]:
NFAT_genes %>% filter()

In [ ]:
# plot valcano plot
EnhancedVolcano::EnhancedVolcano(NFAT_genes,
    lab = NFAT_genes$Gene,
    x = 'value',
    y = 'f1_value_FDR',
    pCutoff = 0.05,
    FCcutoff = 0.6,
    pointSize = 3.0,
    labSize = 6.0) + xlim(-1, 1) + ylim(0, 3) + xlab('weight')
ggsave(file.path(fig_path, paste0(proj_name, '_f1_NFAC_genes_valcano_plot.png')), width = 8, height = 8)

In [ ]:
# run gene enrichment for NFAC famliy target genes
NFAT_genes_sig <- NFAT_genes %>% filter(abs(value)>0.6) %>% pull(Gene) %>%unique()
NFAT_genes_sig %>%length()
enrich_cd4na_NFAT <- enrichr(NFAC3_genes_sig, c('KEGG_2021_Human', 'MSigDB_Hallmark_2020'))
 

In [ ]:
NFAC3_genes_sig

In [ ]:
enrich_cd4na_NFAC3$MSigDB_Hallmark_2020 %>% filter(Adjusted.P.value<0.05) %>% 
    dplyr::select(-c(Old.P.value,Old.Adjusted.P.value))
enrich_cd4na_NFAC3$KEGG_2021_Human %>% filter(Adjusted.P.value<0.05) %>% 
    dplyr::select(-c(Old.P.value,Old.Adjusted.P.value))

## peak to gene correlation

In [ ]:
# load the sample tile matrix
SampleTileMatricesAnnotated <- readRDS(file.path(output_path,
                                        paste0(proj_name, '_scMACS_total_cd4na_SampleTileMatrices.rds')))

In [ ]:
# load the annotate pseudo gex dataset
ra_tea_l2_pseudo <- readRDS(file.path(data_path, 'PreRA_teaseq_nichenet_all_cells_l2_subject_pseudoBulk.rds'
                                ))

In [ ]:
# load the degs for cd4 naive
# load the deg list
ra_tea_l2_pseudo_degs <- 
    read_csv('/home/jupyter/data/preRA_teaseq/output_results/PreRA_teaseq_l2_annotation_PseudoBulk_degs_preRAvsHealthy.csv',
            show_col_types = FALSE) %>%
    filter(!is.na(FDR) & !is.na(logFC)) %>% mutate(direction=if_else(logFC>0, 'up', 'down'))
# filter cd4 naive
cd4na_pseudo_degs <- ra_tea_l2_pseudo_degs %>% filter(cell_type=='cd4_naive')
cd4na_pseudo_degs %>% head()

In [ ]:
cd4na_pseudo_degs %>% nrow()

In [ ]:
# subset the cd4 naive
cd4na_pseudo <- ra_tea_l2_pseudo[cd4na_pseudo_degs$gene, colData(ra_tea_l2_pseudo)$clean_l2_cell_types=='cd4_naive']
# make colnames of the pseudo object as subject id
colnames(cd4na_pseudo) <- colData(cd4na_pseudo)$subject_id
cd4na_pseudo <- scuttle::logNormCounts(cd4na_pseudo)

In [ ]:
cd4na_pseudo_degs %>%  filter(FDR<0.2&gene=='STAT3')

In [ ]:
# get sample level nomralized gene matrix from seurat object
ra_tea_cd4na <- SetIdent(ra_tea_cd4na, value = 'subject_id')
# get the sample average corrected SCT gene expression data for cd4 anive
cd4na_gex_mx <- AverageExpression(ra_tea_cd4na, features = cd4na_pseudo_degs %>% 
                                  filter(FDR<0.05) %>% pull(gene), assays = 'SCT')$SCT
cd4na_gex_mx %>% dim()

### test FigR correlation at Psudobulk

In [ ]:
# try figR
quiet_library(FigR)

In [ ]:
# fill na with 0 in the peak counts
ATAC.se=SampleTileMatricesAnnotated
assay(ATAC.se)[is.na(assay(ATAC.se))] = 0

In [ ]:
colnames(ATAC.se) = colData(ATAC.se)$subject_id

In [ ]:
ATAC.se

In [ ]:
ATAC.se <- chromVAR::addGCBias(ATAC.se,  genome = BSgenome.Hsapiens.UCSC.hg38)
# bg <- chromVAR::getBackgroundPeaks(ATAC.se)

In [ ]:
rowData(ATAC.se) %>% as_tibble(rownames = 'gemone_region') %>% filter(is.na(bias))

In [ ]:
options(repr.plot.width = 6, repr.plot.height = 6)
rowData(ATAC.se) %>% as_tibble(rownames = 'gemone_region') %>% pull(bias) %>% hist()

In [ ]:
counts(ATAC.se) = assay(ATAC.se)

In [ ]:
ATAC.se <- ATAC.se[!is.na(rowData(ATAC.se)$bias), ]

In [ ]:
rowData(ATAC.se)[is.na(rowData(ATAC.se)$bias), ]

In [ ]:
ATAC.se

In [ ]:
# ATAC.se <- chromVAR::getBackgroundPeaks(ATAC.se)

In [ ]:
any(is.na(counts(ATAC.se)))

In [ ]:
cd4na_gex_mx %>% colnames() %>% typeof()

In [ ]:
cd4na_gex_mx %>% colnames()
ATAC.se %>% colnames() %>% str_extract('\\d')

In [ ]:
names(colnames(cd4na_gex_mx)) <-NULL
# ATAC.se

In [ ]:
cd4na_gex_mx <- cd4na_gex_mx[, colnames(ATAC.se)]

In [ ]:
# Run using multiple cores if parallel support
cisCor <- FigR::runGenePeakcorr(ATAC.se=ATAC.se,
  cd4na_gex_mx,
  genome='hg38',
  geneList = NULL,
  windowPadSize = 100000,
  normalizeATACmat = FALSE,
  nCores = 40,
  keepPosCorOnly = FALSE,
  keepMultiMappingPeaks = FALSE,
  n_bg = 100,
  p.cut = NULL)

In [ ]:
cisCor %>% write_csv(file.path(output_path, paste0(proj_name, '_figR_cisCor_degs_correlation.csv')))

In [ ]:
# load correlation results
cisCor <- read_csv('/home/jupyter/data/preRA_teaseq/output_results/cd4_t/preRA_tea_seq_cd4tna_figR_cisCor_degs_correlation.csv')
cisCor.filt <- cisCor %>% filter(pvalZ <= 0.05) %>% dplyr::rename('gene'='Gene') %>%
    mutate(seqnames=str_split(PeakRanges, ':', simplify = TRUE)[,1], 
          start=str_split(PeakRanges, ':|-', simplify = TRUE)[,2],
           end=str_split(PeakRanges, ':|-', simplify = TRUE)[,3])
cisCor.filt %>% head()

In [ ]:
cisCor_granges <- cisCor.filt %>% GenomicRanges::makeGRangesFromDataFrame(keep.extra.columns=TRUE)

In [ ]:
cd4na_dap_cor <- filter_by_overlaps(cisCor_granges, cd4na_diff_sig)

In [ ]:
cd4na_dap_cor %>% filter(str_detect(gene, 'IL\\d|IKZF|CD\\d')) %>% as_tibble()

### run motif enriment for each gene

In [ ]:
# dorcGenes

In [ ]:
# Determine DORC genes
dorcGenes <- cisCor %>% filter(pvalZ <= 0.05)  %>% dorcJPlot(cutoff=5, # Default
                                       returnGeneList = TRUE)
ggsave(file.path(fig_path, paste0(proj_name, '_figR_peak_gene_correlation_jplot.png')), width = 8, height = 6)

In [ ]:
cd4na_diff_bg %>% filter_by_non_overlaps(cisCor.sel)

In [ ]:
# for each genes that have 5 or more peaks correlated run motif enrichment
dorc_motif <- mclapply(dorcGenes, function(dorc_gene){
    cisCor.sel <- cisCor.filt %>% filter(gene == dorc_gene) %>%
        GenomicRanges::makeGRangesFromDataFrame(keep.extra.columns=TRUE)
    cisCor.motif <- MOCHA::MotifEnrichment(cisCor.sel, Group2 = cd4na_diff_bg %>% filter_by_non_overlaps(cisCor.sel), 
                                                  motifPosList=metadata(SampleTileMatricesAnnotated)$Motifs)%>% 
        as_tibble(rownames = 'motif') %>% 
        mutate(sig=if_else(adjp_val<0.05, 'sig', 'not sig'),
              'correlated_gene'=dorc_gene)
    return(cisCor.motif)
}, mc.cores = 58) %>% rbindlist()


In [ ]:
dorc_motif <- dorc_motif %>% dplyr::rename('gene'='correlated_gene')

In [ ]:
dorc_motif %>% head()

In [ ]:
dorc_motif %>% 
    filter(adjp_val<0.05) %>% distinct(motif) %>% nrow()
dorc_motif %>% 
    filter(adjp_val<0.05) %>% distinct(gene
                                      ) %>% nrow()

In [ ]:
dorc_motif %>% 
    filter(adjp_val<0.05) %>% group_by(gene) %>% tally() %>% ggplot(aes(x=n))+geom_histogram()

In [ ]:
# find a list of motif gene asscoiateion that hold true in both
cd4na_tf_gex_cor_fl <- cd4na_tf_gex_cor %>% left_join(dorc_motif, by=c('motif', 'gene')) %>% 
    filter(p.value<0.05&adjp_val<0.05) %>% 
    mutate(corr_direction=if_else(spearman_cor>0, 'pos', 'neg'), 
          motif_gene=paste(motif, gene, sep = '_'))

In [ ]:
cd4na_tf_gex_cor_fl %>% head()
cd4na_tf_gex_cor_fl %>% nrow()

In [ ]:
cd4na_tf_gex_cor_fl %>% ggplot(aes(x=spearman_cor, y=enrichment, label=motif_gene)) +
    geom_point() + ggrepel::geom_text_repel()
ggsave(file.path(fig_path, paste0(proj_name, '_chromVAR_tf_deviation_peaks_gex_pairs_sig.png')),
       width = 8, height = 8)

In [ ]:
motif_gene_cor_counts=cd4na_tf_gex_cor_fl %>% group_by(motif, corr_direction) %>% tally() %>%
    mutate(n=if_else(corr_direction=='neg', -n, n)) 
# motif_gene_cor_counts
motif_gene_cor_counts %>% ggplot(aes(x=n, y=motif, fill=corr_direction)) + geom_col()
ggsave(file.path(fig_path, paste0(proj_name, '_chromVAR_tf_deviation_peaks_gex_correlation_sig.png')),
       width = 6, height = 8)

In [ ]:
enrich_cd4na_tf_gex_cor_fl <- enrichr(cd4na_tf_gex_cor_fl$gene %>% unique(), c('KEGG_2021_Human', 'MSigDB_Hallmark_2020'))


In [ ]:
enrich_cd4na_tf_gex_cor_fl$MSigDB_Hallmark_2020 %>% filter(Adjusted.P.value<0.05)

In [ ]:
# cisCor.filt %>% head()
marker_gene <- c('GATA3', 'RORC', 'RORA','FOXP3', 'TBX21', 'IKZF2', 'STAT3', 'RORA', 'IFNG-AS1', 'CCL5')
cisCor.filt %>% filter(Gene %in% marker_gene) 

In [ ]:
numDorcs <- cisCor.filt %>% group_by(Gene) %>% tally() %>% arrange(desc(n))
numDorcs

In [ ]:
dorcMat <- getDORCScores(ATAC.se = ATAC.se, # Has to be same SE as used in previous step
                         dorcTab = cisCor.filt,
                         geneList = dorcGenes,
                         nCores = 40)

In [ ]:
# dorcMat

In [ ]:
fig.d <- runFigRGRN(ATAC.se=ATAC.se,
                    rnaMat=cd4na_gex_mx, # Smoothed RNA matrix using paired cell kNNs
                    dorcMat=dorcMat,
                    dorcTab=cisCor.filt,
                    genome="hg38",
                    dorcGenes=dorcGenes,
                    nCores=40)

In [ ]:
fig.d

In [ ]:
#' @title \code{PeakGeneLinks}
#'
#' @description \code{PeakGeneLinks} takes an input set of regions (tiles) and finds co-accessible neighboring regions within a window. Co-accessibility is defined as the correlation between two region intensity (openness) across samples.
#'
#' @param SampleTileObj The SummarizedExperiment object output from getSampleTileMatrix containing your sample-tile matrices
#' @param seRNA The SummarizedExperiment object from single cell RNA. The names of the assays should be cell types. Cell type and sample names must match SampleTileObj.
#' @param windowSize the size of the window, in basepairs, around each input region to search for co-accessible links 
#' @param susbetData If samples and cell type annotations don't align across seRNA and SampleTileObj, run correlation across those celltypes/samples that do align. 
#' @param numCores Optional, the number of cores to use with multiprocessing. Default is 1.
#'
#' @return 
#' 
#' @details The technical details of the zero-inflated correlation can be
#'          found here:
#'
#'               Pimentel, Ronald Silva, "Kendall's Tau and Spearman's Rho
#'               for Zero-Inflated Data" (2009). Dissertations.
#'
#'          while the implementation (scHOT R package), can be found here:
#'               http://www.bioconductor.org/packages/release/bioc/html/scHOT.html
#'
#'
#' @export

getPeakGeneLinks <- function(SampleTileObj, seRNA,
                                 specificGenes = NULL, 
                                 windowSize = 1 * 10^6, 
                                 numCores = 1, 
								 subsetData = FALSE,
                                 verbose = FALSE) {
  
  
  ## Pull out the gene list of interest
  geneList <- SummarizedExperiment::rowRanges(seRNA)
  
  if(!is.null(specificGenes)){
	if (class(specificGenes) == "GRanges") {	
		geneDF <- as.data.frame(specificGenes)
	  } else if (class(specificGenes) == "character") {
	  
		geneDF <- plyranges::filter(geneList, gene %in% specificGenes)
		if(length(geneDf) == 0){stop('Those specific genes were not found. Please provide list of gene symbols.')}
		if(length(geneDf) < length(specificGenes)){
			diffGenes <- length(specificGenes) - length(geneDf)
			warning(paste(diffGenes, ' genes not found within seRNA object', sep = ''))
		}
		
		
	  } else {
		stop('Invalid input type for "specificGenes": must be either "GRanges" or a character vector')
	  }
  }else{
  
	geneDF <- as.data.frame(geneList)
  
  }
  
  
  #### Extract matrices for gene expression and tile accessibility
  
  if(! any(names(assays(seRNA)) %in% names(assays(sampleTileObject)))){
  
	stop('No cell type labels align between the seRNA and sampleTileObject')
  
  }else{
  
	tileSamples <- SummarizedExperiment::colData(SampleTileObj)
	geneSamples <- SummarizedExperiment::colData(seRNA)
	
	overlapSamples <- sum(rownames(tileSamples) %in% rownames(geneSamples))
	
	if(overlapSamples == 0){
	
		stop('Sample names between seRNA and SampleTileObj do not match')
	
	}else if((overlapSamples < dim(tileSamples)[1] | overlapSamples < dim(geneSamples)[1])){
	
		
		
		if(subsetData){
		
			warning(paste('Subsetting down to the ', overlapSamples, ' Samples overlap between ' dim(tileSamples)[1] ' scATAC and ', dim(geneSamples)[1], ' scRNA samples', sep = '' ))
			
			
			subsetSTObj <- SampleTileObj[,colnames(SampleTileObj) %in% colnames(geneSamples)]
			subsetRNAObj <- seRNA[,colnames(SampleTileObj) %in% colnames(geneSamples)]
				
		}else{
		
			stop(paste('Only ', overlapSamples, ' Samples overlap between ' dim(tileSamples)[1] ' scATAC and ', dim(geneSamples)[1], ' scRNA samples. Set subsetData = TRUE to run correlation anyways.', sep = '' ))
		
		}
		
	
	}else{
	
	
		subsetSTObj <- SampleTileObj
		subsetRNAObj <- seRNA
	
	}
	
	
	cellTypeOverlap <- 	names(assays(seRNA))[names(assays(subsetSTObj)) %in% names(assays(subsetRNAObj))]
	
	if(!all(names(assays(seRNA)) %in% names(assays(sampleTileObject))) & !subsetData){
	
	
		stop(paste('Not all cell types are found in both seRNA and SampleTileObj. Set subsetData = TRUE to run correlation on only the following celltypes: ', cellTypeOverlap, sep = '' ))
		
	
	}else{
	
		warning(paste('Not all cell types are found in both seRNA and SampleTileObj. Running correlations for the following celltypes: ', cellTypeOverlap, sep = '' ))
		
				
		tileDF <- getFullMatrix(subsetSTObj, cellTypes = cellTypeOverlap) 
	 
		start_tiles <- as.numeric(gsub("chr.*\\:|\\-.*", "", rownames(tileDF)))
		end_tiles <- as.numeric(gsub("chr.*\\:|.*\\-", "", rownames(tileDF)))
		chr <- gsub("\\:.*", "", rownames(tileDF))
	  
		geneExpression <- getFullMatrix(subsetRNAObj, cellTypes = cellTypeOverlap) 
		
	}
	
  }
    

  # Initialize correlation datatable
  PG_Corr <- NULL
  pb <- txtProgressBar(min = 0, max = nrows(geneExpression), initial = 0, style = 3)
  
  for (i in 1:nrows(geneExpression)) {
    setTxtProgressBar(pb, i)
    
    # Find all neighboring tiles in the window
    windowIndexBool <- which(start_tiles > geneDF$start[i] - windowSize / 2 &
                          end_tiles < geneDF$end[i] + windowSize / 2 &
                          chr == geneDF$seqnames[i])
    
    if (length(windowIndexBool) > 0) {
      
      nextCorr <- peakToGene(tileDF[windowIndexBool, drop=FALSE], 
								geneExpression = geneExpression[i,], 
								geneName = geneDF$gene[i],
                                  numCores = numCores,
                                  verbose = verbose
      )
      # For first iteration, PG_Corr is NULL so it will be ignored
      PG_Corr <- rbind(PG_Corr, nextCorr)
    } else if (verbose) {
      warning(
        "No neighboring tiles found for given gene: ",
        stringr::str_interp(
         "${geneDF$gene[i]} at ${geneDF$seqnames[i]}:${geneDF$start[i]}-${geneDF$end[i]}"
        ),
        stringr::str_interp(" with windowSize ${windowSize} basepairs")
      )
    }
  }
  
  close(pb)
  PG_Corr
}


#' @title \code{co_accessibility}
#'
#' @description \code{co_accessibility} allows you to determine whether 2 tiles
#'              are co-accessible using a zero-inflated spearman correlation.
#'
#'
#' @param mat sample-tile matrix with regions to analyze
#' @param numCores integer to determine # of parallel cores
#'
#' @return a 3-column data.frame containing
#'         - Correlation = Zero-inflated Spearman Correlation
#'         - Tile1 = location of co-accessible region 1
#'         - Tile2 = location of co-accessible region 2
#'
#' @details The technical details of the zero-inflated correlation can be
#'          found here:
#'
#'               Pimentel, Ronald Silva, "Kendall's Tau and Spearman's Rho
#'               for Zero-Inflated Data" (2009). Dissertations.
#'
#'          while the implementation (scHOT R package), can be found here:
#'               http://www.bioconductor.org/packages/release/bioc/html/scHOT.html
#'
#' @references XX
#' @examples
#' Generate
#' mat1 <- matrix(pmax(0, rnorm(1000)), ncol = 100)
#' row.names(mat1) <- paste("A", 1:10, sep = "_")
#' ziSpear_mat <- co_accessibility(mat1, numCores = 5)
#' head(ziSpear_mat)
#'
#' @internal
#' @noRd

peakToGene <- function(subTileMat, geneExpression, geneName, numCores = 1, verbose = FALSE) {
 
  # If only one pair of tiles left, then nrows will hit an error.
  # length()==2 is equivalent to one row (pair)
  if (dim(subTileMat)[1] == 0 & verbose) {
    warning("Warning: No tiles found in neighborhood of region of interest")
    return(NULL)
  } else if (dim(subTileMat)[1] == 0) {
    return(NULL)
  } else if (dim(subTileMat)[1] == 1) {
    
    # If only one pair of tiles to test, then it's no longer a data.frame, but a vector.
    zero_inflated_spearman <- scMACS:::weightedZISpearman(
      x = geneExpression,
      y = subTileMat[1,]
    )

    zi_spear_mat <- data.table(
      Correlation = zero_inflated_spearman,
      Gene = geneName,
      Tile = rownames(subTileMat)
    )
  } else {
    
    # General case for >1 pair
    zero_inflated_spearman <- unlist(parallel::mclapply(1:nrow(subTileMat),
      function(x) {
        scMACS:::weightedZISpearman(
          x = geneExpression,
          y = subTileMat[x,]
        )
      },
      mc.cores = numCores
    ))

    # Create zero-inflated correlation matrix from correlation values
    zi_spear_mat <- data.table(
      Correlation = zero_inflated_spearman,
      Gene = rep(geneName, length( zero_inflated_spearman)),
      Tile = rownames(subTileMat)
    )
  }

  return(zi_spear_mat)
}

#######################################################################################
#' scHOT implemented Pimentel's zero-inflated (ZI) correlation
#' in their R package, providing implementations
#' of the ZI spearman and tau rank correlations.
#' scMACS implements a slight modification of
#' scHOT's zero-inflated correlation measure, by
#' returning NAs in the cases where the correlation
#' is undefined, and modifying it to use a C-backed correlation program.
#' Both references are provided below and are
#' referenced in all documentations to indicate
#' their work in implementing these methods.
#'
#'
#' Description:
#' the weightedZISpearman function calculates weighted rho\*,
#' where rho\* is described in Pimentel et al (2009).
#' This association measure is defined for zero-inflated,
#' non-negative random variables.
#'
#' @title weightedZISpearman
#' @param w weight vector, values should be between 0 and 1
#' @param x x and y are non-negative data vectors
#' @param y x and y are non-negative data vectors
#' @return \code{numeric} weighted rho* association value between x and y
#'
#'
#' @references scHOT
#' Ghazanfar, S., Lin, Y., Su, X. et al. Investigating higher-order interactions in
#'   single-cell data with scHOT. Nat Methods 17, 799–806 (2020).
#'   https://doi.org/10.1038/s41592-020-0885-x
#'
#' Zero-Inflated Correlation
#' Pimentel, Ronald Silva, "Kendall's Tau and Spearman's Rho for
#'   Zero-Inflated Data" (2009). Dissertations. 721.
#'   https://scholarworks.wmich.edu/dissertations/721
#'
#' @internal
#' @noRd

weightedZISpearman <- function(x, y, w = 1) {

  # needs the original values, not the ranks

  if (any(x < 0 | y < 0)) {
    stop("x and/or y values have negative values")
  }
  if (length(x) != length(y)) {
    stop("x and y should have the same length")
  }
  if (length(w) == 1) {
    w <- rep(w, length(x))
  }

  posx <- x > 0
  posy <- y > 0
  pospos <- posx & posy

  p_11 <- sum(w * pospos) / sum(w)
  p_00 <- sum(w * (!posx & !posy)) / sum(w)
  p_01 <- sum(w * (!posx & posy)) / sum(w)
  p_10 <- sum(w * (posx & !posy)) / sum(w)


  if (any(pospos) & p_11 > 0) {
    rho_11 <- wCorr::weightedCorr(x = x[pospos], y = y[pospos], weights = w[pospos], method = "Spearman")
  } else {
    print("Zero inflated Spearman correlation is undefined,
          returning NA")
    rho <- NA
    return(rho)
  }

  rho_star <- p_11 * (p_01 + p_11) * (p_10 + p_11) * rho_11 +
    3 * (p_00 * p_11 - p_10 * p_01)

  if (is.na(rho_star)) {
    print("Zero inflated Spearman correlation is undefined,
          returning NA")
    rho <- NA
    return(rho)
  }



  return(rho_star)
}


# Extract a combined matrix of sample-cell type pseudobulks, with cell types and samples sorted.
getFullMatrix <- function(SampleTileObj, cellTypes){

	#Extract all assays
	sortedSamples <- lapply(as.list(assays(sampleTileObject)), function(x){
	
					x[,sort(colnames(x))]
			}
	names(subAssay) <- names(assays(SampleTileObj))

	#subset to just populations of interest
	subAssay <- sortedSamples[names(assays(SampleTileObj)) %in% cellTypes]

	#Sort cell types so that they're in the same order
	sortedAssays <- subAssay[sort(names(subAssay))]
	
	#Column-bind the list of data.frames into one combined matrix
	combMat <- do.call('cbind', sortedAssays)
	
	return(combMat)
s
}

## MOTIF analysis

### motif enrichment by chromVAR by cell types

In [ ]:
library(chromVAR)
library(motifmatchr)
library(BSgenome.Hsapiens.UCSC.hg38)
library(BiocParallel)
set.seed(2017)
library(Matrix)

In [ ]:
register(MulticoreParam(60, progressbar = TRUE))

In [ ]:
#Getting GC content of peaks
SampleTileMatrices <- addGCBias(SampleTileMatrices, 
                            genome = BSgenome.Hsapiens.UCSC.hg38)

In [ ]:
cd4na_diff_sig <- cd4na_diff[(!is.na(elementMetadata(cd4na_diff)[,'FDR'])) & 
                             (elementMetadata(cd4na_diff)[,'FDR']>0.2) & 
                             (elementMetadata(cd4na_diff)[,'Log2FC_C']>0.1)]
cd4na_diff_sig

In [ ]:
# motif_enrichment
# filter significant daps
cd4na_diff_bg <- cd4na_diff[(!is.na(elementMetadata(cd4na_diff)[,'FDR'])) & 
                             (elementMetadata(cd4na_diff)[,'FDR']>0.2)]
cd4na_dap_sig_up <- cd4na_diff[(!is.na(elementMetadata(cd4na_diff)[,'FDR'])) & 
                             (elementMetadata(cd4na_diff)[,'FDR']<0.2) & 
                             (elementMetadata(cd4na_diff)[,'MeanDiff']>0)]
cd4na_up_motif <- MOCHA::MotifEnrichment(cd4na_dap_sig_up, Group2 = cd4na_diff_bg, 
                                                  motifPosList=metadata(SampleTileMatricesAnnotated)$Motifs)%>% 
    as_tibble(rownames = 'motif') %>% 
    mutate(sig=if_else(adjp_val<0.05, 'sig', 'not sig'))


In [ ]:
# motif down 
cd4na_dap_sig_down <- cd4na_diff[(!is.na(elementMetadata(cd4na_diff)[,'FDR'])) & 
                             (elementMetadata(cd4na_diff)[,'FDR']<0.2) & 
                             (elementMetadata(cd4na_diff)[,'MeanDiff']<0)]
cd4na_down_motif <- 
    MOCHA::MotifEnrichment(cd4na_dap_sig_down,
                           Group2 = cd4na_diff_bg, 
                           motifPosList=metadata(SampleTileMatricesAnnotated)$Motifs)%>% 
    as_tibble(rownames = 'motif')%>% 
    mutate(sig=if_else(adjp_val<0.05, 'sig', 'not sig'))


In [ ]:
# cd4na_dap_sig_up

In [ ]:
cd4na_sig_up_motif <- cd4na_diff_sig_up_motif %>% as_tibble(rownames = 'motif') %>% 
    filter(adjp_val<0.05)

In [ ]:
# cd4na_diff$MeanDiff %>% unique()

In [ ]:
p1 <- cd4na_up_motif %>%
    ggplot(aes(x=enrichment, y=mlog10Padj, color=sig)) +
    geom_point() + ggrepel::geom_label_repel(aes(label=motif)) +
    ggtitle('motif enriched in peaks up preRA')
p2 <- cd4na_down_motif  %>%
    ggplot(aes(x=enrichment, y=mlog10Padj, color=sig)) +
    geom_point() + ggrepel::geom_label_repel(aes(label=motif)) +
    ggtitle('motif enriched in peaks up Healthy')
p1+p2
ggsave(file.path(fig_path, paste0(proj_name, 'peaks_motif_enriched.png')), width = 12, height = 6)

In [ ]:
cd4na_up_motif %>% filter(str_detect(motif, 'STAT|FOXP|ROR'))

In [ ]:
# Filtering inputs
# find indices of samples to keep
sampleTileMatrics_fl <- filterSamples(SampleTileMatrices, min_depth = 1500, 
                                 min_in_peaks = 0.15, shiny = FALSE)

In [ ]:
sampleTileMa

In [ ]:
# create a assay for cd4 naive
sampleTileMatrics_cd4na <- SummarizedExperiment(assays=list(counts=assay(sampleTileMatrics, 'cd4_memory')),
                     #rowData=rowData(sampleTileMatrics), 
                                                rowRanges=rowRanges(sampleTileMatrics),
                     colData=colData(sampleTileMatrics),
                     metadata=metadata(sampleTileMatrics))


In [ ]:
# Filtering inputs
#find indices of samples to keep
# sampleTileMatrics_cd4na <- filterSamples(sampleTileMatrics_cd4na, min_depth = 1500, 
#                                  min_in_peaks = 0.15, shiny = FALSE)
# sampleTileMatrics_cd4na <- chromVAR::filterPeaks(sampleTileMatrics_cd4na, non_overlapping = TRUE)


In [ ]:
sampleTileMatrics_cd4na_fl <- filterPeaks(sampleTileMatrics_cd4na, non_overlapping = TRUE)

In [ ]:
sampleTileMatrics_cd4na_fl

In [ ]:
motifs <- getJasparMotifs()
motif_cd4na <- matchMotifs(motifs, sampleTileMatrics_cd4na_fl, 
                        genome = BSgenome.Hsapiens.UCSC.hg38)

In [ ]:
bg <- getBackgroundPeaks(object = sampleTileMatrics_cd4na_fl)

motif_cd4na_dev <- computeDeviations(object = sampleTileMatrics_cd4na_fl, annotations = motif_cd4na,
                                    background_peaks = bg)


In [ ]:
variability <- computeVariability(motif_cd4na_dev)
plotVariability(variability, use_plotly = FALSE)

In [ ]:
row_anno <- colData(motif_cd4na_dev)[, c('subject_id','cohort')] %>% as.data.frame()
row.names(row_anno) <- row.names(colData(motif_cd4na_dev))
row_anno

In [ ]:
sample_cor <- getSampleCorrelation(motif_cd4na_dev)

library(pheatmap)
pheatmap(as.dist(sample_cor), 
         annotation_row = row_anno,
         clustering_distance_rows = as.dist(1-sample_cor), 
         clustering_distance_cols = as.dist(1-sample_cor)
        )

In [ ]:
diff_acc <- differentialDeviations(motif_cd4na_dev, "cohort")
diff_acc %>% filter(p_value<0.01)

In [ ]:
# add the tile information to the differentail results
tile_info <- rowData(SampleTileMatricesAnnotated) %>% as.data.table(keep.rownames = 'Tile')
differentials <- differentials %>% left_join(tile_info, by='Tile')

In [ ]:
# save the differential results
differentials %>% fwrite(file.path(output_path, 
                                  paste0(proj_name, '_scMACS_DAPs_RAvsHealthy.tsv')))

In [ ]:
# save the differential results
differentials <- fread(file.path(output_path, 
                                  paste0(proj_name, '_scMACS_DAPs_RAvsHealthy.tsv')))

In [ ]:
differentials %>% filter(!is.na(FDR))  %>% ggplot() + geom_histogram(aes(FDR))

In [ ]:
differentials %>% filter(FDR<0.2 & !is.na(Gene))

### GREAT analysis

In [ ]:
library(rGREAT)

In [ ]:
# get the differntial peaks and the background peaks
cd4na_dap <- cd4na_diff[(!is.na(elementMetadata(cd4na_diff)[,'FDR'])) & (elementMetadata(cd4na_diff)[,'FDR']<0.2)]
cd4na_bgp <- cd4na_diff[(!is.na(elementMetadata(cd4na_diff)[,'FDR'])) & (elementMetadata(cd4na_diff)[,'FDR']>0.2)]

In [ ]:
job = submitGreatJob(gr = cd4na_dap, bg = cd4na_diff, species = "hg38")

In [ ]:
job

In [ ]:
tb = getEnrichmentTables(job)

In [ ]:
tb2 = getEnrichmentTables(job, download_by = "tsv")

In [ ]:
availableOntologies(job)

In [ ]:
res = plotRegionGeneAssociationGraphs(job)

In [ ]:
res[1:10, ]

### Motif enrichment

In [ ]:
##########
### Functions for motif analysis (enrichment and chromVar)
#' @title \code{addMotifSet}
#'
#' @description \code{addMotifSet}Identify motifs within peakset
#'
#' @param SE_Object your scMACS SummarizedExperiment. Requires Genome AnnotationDbi object within the metadata added by getSampleTileMatrix
#' @param pwms a pwms object for the motif database. Either PFMatrix, PFMatrixList, PWMatrix, or PWMatrixLis'
#' @param w the width for motifmatchr
#' @param returnObj if TRUE, return the modified SE_Object with motif set added to metadata (default). If FALSE, return the motifs from motifmatchr.
#' @param motifSetName name of the motifList in the SE_object's metadata if returnObj=TRUE. Default is 'Motifs'.
#'
#' @return the modified SE_Object with motifs added to the metadata
#' @examples
#' \dontrun{
#' # load a curated motif set from library(chromVARmotifs) included with ArchR installation
#' data(human_pwms_v2)
#' SE_with_motifs <- addMotifSet(SE_Object, pwms = human_pwms_v2, returnObj = TRUE, motifSetName = "Motifs", w = 7)
#' }
#'
#' @export

addMotifSet <- function(SE_Object, pwms, w = 7, 
                        returnObj = TRUE, motifSetName = "Motifs") {
  TotalPeakSet <- rowRanges(SE_Object)
  genome <- metadata(SE_Object)$Genome
  motif_ix <- motifmatchr::matchMotifs(
    pwms = pwms,
    TotalPeakSet,
    genome = genome,
    out = "positions", w = w
  )
  names(motif_ix) <- sub("_D_.*|_I_.*", "", names(motif_ix)) %>%
    sub("_I$|_D$", "", .) %>%
    sub(".*_LINE", "", .) %>%
    sub(".*_", "", .)

  motifList <- list(motif_ix)
  names(motifList) <- motifSetName

  if (returnObj) {
    metadata(SE_Object) <- append(metadata(SE_Object), motifList)
    return(SE_Object)
  } else {
    return(motif_ix)
  }
}


## runChromVar is a wrapper for chromVAR from scMACS
## Obj could be a ragged experiment or a sumarized experiment
## motifGRangesList is a list of all motif positions in a GRangesList format
## genome is the reference genome we use


runChromVar <- function(Obj, motifs,
                       genome =  BSgenome.Hsapiens.UCSC.hg38){
    
    if(class(Obj)[1] == "RaggedSummarizedExperiment" & class(motifs)[1]=='GRangesList'){
        
        Obj1 <- RaggedExperiment::compactSummarizedExperiment(RepTiles, i = 'TotalIntensity')
        tmp <- SummarizedExperiment::assays(Obj1)
        tmp[[1]][is.na(tmp[[1]])] = 0
        names(tmp) <- "counts"
        assays(Obj1) <- tmp

	motifGRangesList = motifs
        
    }else if(class(Obj)[1] == "RangedSummarizedExperiment"){
        
        if( !names(assays(Obj)) %in% 'counts'){
        
		
            
        }else if(class(motifs)[1] == 'GRangesList'){
            
            Obj1 = Obj
	    motifGRangesList = motifs
	    
        }else if(class(motifs)[1] == 'character' & any(names(metadata(Obj)) %in% motifs)){
            
            Obj1 = Obj
	    motifGRangesList = metadata(Obj)[[motifs]]
	    
        }else{

	    stop('Error: No motifset found. Check input')
	}
        
    }else{
    
        stop('Error: Wrong Input Object. Must be either a RaggedExperiment or a RangedSummarizedExperiment')
    }     

    CisbpAnno <- chromVAR::getAnnotations(motifGRangesList, rowRanges = rowRanges(Obj1))

    Obj1 <- chromVAR::addGCBias(Obj1, genome = genome)
    
    dev <- chromVAR::computeDeviations(object = Obj1, 
                         annotations = CisbpAnno)
    
    return(dev)
}



#################      Category          Not in Category       Total
## Group1      length(Group1Cat)        length(OnlyGroup1)       m 
## Group2      length(Group2Cat)        length(OnlyGroup2)       n
# Total                  k
                 
### Enrichment test for GRanges. Test for enrichment of Category within Group1
### @Group1 - A GRanges object for one set of positions
### @Group2 - The background GRanges object, non-overlapping with Group1.
### @Category - A GRanges object of known locations, such as motifs, that you want to test for enrichment in Group1.
### @type - Default is null. You can use this to pull out or simplify the test to a metadata column within the GRanges 
###          for Group1 and Group2. For example, if you want to test for enrichment of all genes, instead of open regions. 
###          If type = null, then it will just use the number of Ranges instead of the number of unique 
###           entries in column 'type'
EnrichedRanges <- function(Group1, Group2, Category, type = NULL, returnTable = FALSE){
    
    Group1Cat <- plyranges::filter_by_overlaps(Group1, Category) 
    Group2Cat <- plyranges::filter_by_overlaps(Group2, Category)
   
    OnlyGroup1 <- plyranges::filter_by_non_overlaps(Group1, Category)
    OnlyGroup2 <- plyranges::filter_by_non_overlaps(Group2, Category)
    
    if(returnTable & is.null(type)){

        dt_table <- data.frame(Group1 = c(length(Group1Cat), length(OnlyGroup1)), 
                               Group2 = c(length(Group2Cat), length(OnlyGroup2)), 
                       row.names = c('In Category', 'Not in Category')) 
    
        return(t(dt_table))
        
    }else if(returnTable & 
             sum(c(colnames(mcols(Group1)),colnames(mcols(Group2))) %in% type) == 2 &
             length(type) == 1){
        
       dt_table <- data.frame(Group1 = c(length(unique(GenomicRanges::mcols(Group1Cat)[,type])), 
                                          length(unique(GenomicRanges::mcols(OnlyGroup1)[,type]))), 
                               Group2 = c(length(unique(GenomicRanges::mcols(Group2Cat)[,type])), 
                                         length(unique(GenomicRanges::mcols(OnlyGroup2)[,type]))), 
                       row.names = c('In Category', 'Not in Category')) 
    
        return(t(dt_table))
       
    }else if(returnTable){
        
        stop('Error: Incorrect method or column name. Please check input')      
    }
    
    if(is.null(type)){
        
       pVal <- phyper(q = length(Group1Cat), 
           m = length(Group1), 
           n = length(Group2),
           k = length(Group1Cat) + length(Group2Cat),
            lower.tail=FALSE)

       enrichment <- (length(Group1Cat)/length(Group1))/(length(Group2Cat)/length(Group2))
        
    }else if(sum(c(colnames(mcols(Group1)),colnames(mcols(Group2))) %in% type) == 2 &
             length(type) == 1){
        
       pVal <- phyper(q = length(unique(mcols(Group1Cat)[,type])), 
           m = length(unique(mcols(Group1)[,type])), 
           n = length(unique(mcols(Group2)[,type])),
           k = length(unique(mcols(Group1Cat)[,type])) + length(unique(mcols(Group2Cat)[,type])),
                     lower.tail=FALSE)
        
       enrichment <- (length(unique(mcols(Group1Cat)[,type]))/length(unique(mcols(Group1)[,type])))/
                    (length(unique(mcols(Group2Cat)[,type]))/length(unique(mcols(Group2)[,type])))
        
    }else{
        
        stop('Error: Incorrect method or column name. Please check input')
        
    }
    
    return(data.frame(p_value = pVal, enrichment = enrichment))
    
}
    
######## Test all motifs for enrichment.
    

MotifEnrichment <- function(Group1, Group2, motifPosList, type = NULL, numCores = 1){
    
    
    allEnrichmentList <- mclapply(motifPosList, function(x){
        
        tmp_df <- EnrichedRanges(Group1, Group2, Category = x, type = type)
        
    }, mc.cores = numCores)
    df_final <- do.call('rbind',  allEnrichmentList)

    df_final$adjp_val <- p.adjust(df_final$p_value)
    df_final$mlog10Padj <- -log10(df_final$adjp_val)
    
    return(df_final)

}
    
############# Pull out all the motifs associated with each gene according to a set of TSS sites

## @TSS_Sites - GRanges objects that are the list of TSS sites of interest. 
##              Must include a column 'name' which has the associated gene name
## @allPeaks - GRanges object of all peaks 
## @TSS_Links - a data.table object that record all the peak-peak links by co-accessibility
##              Must include columns named 'Peak1' and 'Peak2' which contain a string describing
##              each peak in the format 'chr1:100-2000' and must be identical to peaks listed in
##              allPeaks
## @motifPosList - a GRangesList, which each index is a GRanges of all positions 
##                  for a given motif. GRangesList must be named. 
## @numCores - number of cores to multithread over. 
    
    
Gene2Motif <- function(TSS_Sites, allPeaks, TSS_Links, motifPosList, 
                       numCores = 1, verbose = FALSE){
    
    if(verbose){ print('Generating TSS-Peak Network.')}
    
    TSS_Network <- c(TSS_Links$Peak1, TSS_Links$Peak2, 
                           GRangesToString(TSS_Sites)) %>%
                   unique() %>%
                  StringsToGRanges(.) %>% 
                plyranges::filter_by_overlaps(allPeaks, .)
    
    if(verbose){ print('Finding all motifs related to each peak within the TSS-Peak Network.')}
    ##Let's find all the motifs that overlap with each peak within the altTSS Network
    tmpOverlap <- mclapply(seq_along(motifPosList), function(x){
    
        ifelse(count_overlaps(TSS_Network, motifPosList[[x]]) > 0,
           names(motifPosList)[x], NA)
    
    }, mc.cores= numCores)
    
    
    overlap_df <- do.call('cbind', tmpOverlap)
    motifList <- mclapply(c(1:dim(overlap_df)[1]), function(x){
        
        
        ifelse(any(!is.na(overlap_df[x,])),
            list(overlap_df[x,which(!is.na(overlap_df[x,]))]),
            NA)
    
    }, mc.cores= numCores)

    if(verbose){ print('Finding all peaks related to each gene within the TSS-Peak Network.')}
    ##Find all the peaks related to each gene. 
    
    Peak2Gene <- mclapply(unique(TSS_Sites$name), function(x){
    
        geneTSS <- plyranges::filter(TSS_Sites, name == x)  %>% 
                plyranges::filter_by_overlaps(allPeaks, .) %>%
            plyranges::ungroup() %>%
            GRangesToString(.)
    
        tmp <- TSS_Links[Peak1 %in% geneTSS | Peak2 %in% geneTSS,]
    
        if(dim(tmp)[1] > 0){ 
            unique(c(tmp$Peak1, tmp$Peak2, geneTSS))
        }else{
               geneTSS
        }
    
    }, mc.cores = numCores)
    names(Peak2Gene) <- unique(TSS_Sites$name)
    
    

    if(verbose){ print('Linking Motifs to each gene within the TSS-Peak Network')}
    ## Link all the genes to motifs via Peak2Gene and the motifList
    Gene2Motif <- mclapply(Peak2Gene, function(x){
    
        #Find which indices of the AltTSS Network GRanges are linked to that Gene
        tmp <- findOverlaps(StringsToGRanges(x), TSS_Network)
        #Pull up and unlist all the motifs associated with those tiles.
        unlist(motifList[subjectHits(tmp)])
    
    }, mc.cores = numCores)
    
    return(Gene2Motif)
    
}
    
    

In [ ]:
# motif annotation by chormvar database
library(chromVARmotifs)
library(TFBSTools)
data("human_pwms_v2")
SampleTileMatricesAnnotated <- MOCHA::addMotifSet(SampleTileMatricesAnnotated,
                                                      motifPWMs = human_pwms_v2,
                                                  returnSTM = TRUE,
                                         motifSetName = "Motifs", w = 7)

In [ ]:
SampleTileMatricesAnnotated

In [ ]:
SampleTileMatricesAnnotated %>% saveRDS(file.path(output_path, 
                                  paste0(proj_name, '_scMACS_total_cd4na_SampleTileMatrices.rds')))

In [ ]:
SampleTileMatricesAnnotated <- readRDS(file.path(output_path, 
                                  paste0(proj_name, '_scMACS_total_cd4na_SampleTileMatrices.rds')))

In [ ]:
cd4na_diff

In [ ]:
# run motif enrichment between the 
differentials_ra_up <- cd4na_diff %>% plyranges::filter(FDR<0.2&Log2FC_C>0.1)
differentials_ra_down <- cd4na_diff %>% plyranges::filter(FDR<0.2& Log2FC_C< -0.1)
background <- cd4na_diff %>% plyranges::filter(FDR>0.2)

In [ ]:
differentials_ra_up

In [ ]:
# run motif enrich
up_motif_ravshealthy <- MOCHA::MotifEnrichment(Group1 = differentials_ra_up, Group2 = background, 
                motifPosList = metadata(SampleTileMatricesAnnotated)$Motifs#, numCores = 60
                                                  ) %>%
    as_tibble(rownames = 'motif') 

In [ ]:
# plot valcano plot
EnhancedVolcano::EnhancedVolcano(up_motif_ravshealthy,
    lab = up_motif_ravshealthy$motif,
    x = 'enrichment',
    y = 'adjp_val')
ggsave(file.path(fig_path, paste0(proj_name, '_dap_up_enriched_tf.pdf')), width = 12, height = 12)

In [ ]:
# run motif enrich
differentials_motif_radown <- MotifEnrichment(Group1 = differentials_ra_down, Group2 = background, 
                motifPosList = metadata(SampleTileMatricesAnnotated)$Motifs#, numCores = 60
                                             ) %>%
    as_tibble(rownames = 'motif') 

In [ ]:
# plot valcano plot
EnhancedVolcano::EnhancedVolcano(differentials_motif_radown,
    lab = differentials_motif_ravshealthy$motif,
    x = 'enrichment',
    y = 'adjp_val')
ggsave(file.path(fig_path, paste0(proj_name, '_dap_enriched_tf_down.pdf')), width = 12, height = 12)

In [ ]:
# save the motif enrichment result
differentials_motif_ravshealthy %>%head()
differentials_motif_ravshealthy %>% write_tsv(file.path(output_path, 
                                  paste0(proj_name, '_mocha_ravshealthy_motif_enrichment.tsv')))

In [ ]:
differentials_motif_ravshealthy %>% filter(adjp_val<0.05)

### run chromvar

In [ ]:
# # run chromvar
# chromvar_ravshealthy <- runChromVar(SampleTileMatricesAnnotated, 
#                 motifs = metadata(SampleTileMatricesAnnotated)$Motifs, genome =  BSgenome.Hsapiens.UCSC.hg38) 

In [ ]:
# Obj=SampleTileMatricesAnnotated
# motifs=metadata(SampleTileMatricesAnnotated)$Motifs
# class(Obj)[1] == "RangedSummarizedExperiment" & class(motifs)[1]=='CompressedGRangesList'

In [ ]:
# # run chromvar deviation
# motifGRangesList = metadata(SampleTileMatricesAnnotated)$Motifs

# CisbpAnno <- chromVAR::getAnnotations(motifGRangesList, rowRanges = rowRanges(Obj))

# Obj <- chromVAR::addGCBias(Obj, genome = BSgenome.Hsapiens.UCSC.hg38)
    
# dev <- chromVAR::computeDeviations(object = Obj, 
#                          annotations = CisbpAnno)
    

In [ ]:
# SampleTileMatrices_dev <- runChromVar(SampleTileMatricesAnnotated, motifs =  SampleTileMatrices_motifs,
#                                       genome =  BSgenome.Hsapiens.UCSC.hg38)

In [ ]:
bg <- getBgdPeaks(
  ArchRProj = cd4na_atac,
  nIterations = 50,
  w = 0.1,
  binSize = 50,
  seed = 1,
  method = "chromVAR",
  force = FALSE
)


In [ ]:
bg

In [ ]:

# SampleTileMatricesAnnotated <- chromVAR::addGCBias(SampleTileMatricesAnnotated, genome = BSgenome.Hsapiens.UCSC.hg38)

# dev <- chromVAR::computeDeviations(object = SampleTileMatricesAnnotated, 
#                          annotations = CisbpAnno)

In [ ]:
SampleTileMatricesAnnotated

In [ ]:
assayNames(SampleTileMatricesAnnotated)
assay(SampleTileMatricesAnnotated, 'counts') <- assay(SampleTileMatricesAnnotated, 'cd4_naive')

In [ ]:
assay(SampleTileMatricesAnnotated, 'counts')[1:5, 1:5]

In [ ]:
# change NA to 0 to run Chromvar
assay(SampleTileMatricesAnnotated, 'counts')[is.na(assay(SampleTileMatricesAnnotated, 'counts'))] = 0

In [ ]:
assay(SampleTileMatricesAnnotated, 'counts')[1:5, 1:5]

In [ ]:
SampleTileMatricesAnnotated <- chromVAR::addGCBias(SampleTileMatricesAnnotated, 
                                                   genome = BSgenome.Hsapiens.UCSC.hg38)

In [ ]:
bg <- getBackgroundPeaks(object = SampleTileMatricesAnnotated)

In [ ]:
SampleTileMatricesAnnotated

In [ ]:
library(chromVAR)
library(chromVARmotifs)
data(human_pwms_v2)

library(BSgenome.Hsapiens.UCSC.hg38)
library(SummarizedExperiment)
library(tidyverse)
library(plyranges)

In [ ]:
# assay(SampleTileMatricesAnnotated_fl, 'counts') <- assay(SampleTileMatricesAnnotated_fl, 'cd4_naive')

In [ ]:
# SampleTileMatricesAnnotated_fl
# assay(SampleTileMatricesAnnotated_fl, 'counts')[is.na(assay(SampleTileMatricesAnnotated_fl, 'counts'))] = 0
# SampleTileMatricesAnnotated_fl <- chromVAR::addGCBias(SampleTileMatricesAnnotated_fl, 
#                                                    genome = BSgenome.Hsapiens.UCSC.hg38)

In [ ]:
# get mofit annotation
CisbpAnno <- chromVAR::getAnnotations(metadata(SampleTileMatricesAnnotated)$Motifs, 
                                      rowRanges = rowRanges(SampleTileMatricesAnnotated))

In [ ]:
CisbpAnno

In [ ]:
# assayList <- getCellPopMatrix(SampleTileMatricesAnnotated, 'cd4_naive')
        
# Obj1 <- SummarizedExperiment(
#     assays = list('counts' = assayList),
#     colData = colData(SampleTileMatricesAnnotated),
#     rowRanges = rowRanges(SampleTileMatricesAnnotated),
#     metadata = SampleTileMatricesAnnotated@metadata
# )

# CisbpAnno <- chromVAR::getAnnotations(SampleTileMatricesAnnotated@metadata$Motifs,
#                                       rowRanges = rowRanges(Obj1))

# Obj1 <- chromVAR::addGCBias(Obj1, genome = BSgenome.Hsapiens.UCSC.hg38)
    
# dev <- chromVAR::computeDeviations(object = Obj1,
#                          annotations = CisbpAnno)

In [ ]:
# run the diviation scores
cd4na_dev <- chromVAR::computeDeviations(object = SampleTileMatricesAnnotated, 
                         annotations = CisbpAnno)

In [ ]:
cd4na_dev
variability <- computeVariability(cd4na_dev)
plotVariability(variability, use_plotly = FALSE)

In [ ]:
colnames(cd4na_dev) = colData(cd4na_dev)$subject_id

In [ ]:
cd4na_dev

In [ ]:
cd4na_dev %>% saveRDS(file.path(output_path, paste0(proj_name, 'DAP_chromvar_deviation_rmBR2024.rds')))

In [ ]:
paste0(proj_name, 'DAP_chromvar_deviation_rmBR2024.rds')

In [ ]:
# load chromvar motif deviation in cd4 naive
cd4na_dev <- readRDS(file.path(output_path, paste0(proj_name, 'DAP_chromvar_deviation_rmBR2024.rds')))

In [ ]:
# remove one motif with empty values
cd4na_dev <- cd4na_dev[rownames(cd4na_dev)!='ENSG00000250542',]


In [ ]:
tf_select <- colnames(CisbpAnno) %>% str_subset('NFAT|STAT|JUN|FOS|BCL6|BATF')
tf_select

In [ ]:
# check motif synergy
# tf_select <- colnames(CisbpAnno) %>% str_subset('NFAT|STAT|JUN|FOS|BCL6|BATF|IRF')
nfac_syn <- getAnnotationSynergy(SampleTileMatricesAnnotated, CisbpAnno[, tf_select])

In [ ]:
nfac_syn[, c('NFATC3','NFATC1', 'NFATC2')]

In [ ]:
png(file.path(fig_path, paste0(proj_name, '_NFATs_Synergy_TFs.png')), , units = 'in',res=300,
    width = 8, height = 4)
p1 <- ComplexHeatmap::Heatmap(nfac_syn[, c('NFATC3','NFATC1', 'NFATC2',  'NFAT5',  'NFATC4')])
p1
dev.off()

In [ ]:
p1

In [ ]:
nfac_cor <- getAnnotationCorrelation(SampleTileMatricesAnnotated_fl, CisbpAnno[, tf_select])

In [ ]:
nfac_cor

In [ ]:

png(file.path(fig_path, paste0(proj_name, '_NFATs_correlation_heatmap_TFs.png')), , units = 'in',res=300,
    width = 6, height = 4)
p1=ComplexHeatmap::Heatmap(nfac_cor[, c('NFATC3','NFATC1', 'NFATC2',  'NFAT5',  'NFATC4')])
p1
dev.off()

In [ ]:
tf_select <- colnames(CisbpAnno) %>% str_subset('NFAT|STAT|JUN|FOS|BCL6|BATF')
CisbpAnno[, tf_select]

In [ ]:
# extract the deviation score
cd4na_dev_score <- deviations(cd4na_dev) %>% 
    as_tibble(rownames = 'motif') %>% 
    pivot_longer(cols = -motif, names_to = 'subject_id', values_to = 'deviation_score') %>%
    left_join(as_tibble(colData(cd4na_dev)), by='subject_id') %>% 
    mutate(cohort=factor(cohort, levels = c('Healthy', 'pre-RA')))
cd4na_dev_score %>% head()

In [ ]:
cd4na_dev_score %>% distinct(cohort)

In [ ]:
suppressPackageStartupMessages(library('rstatix'))

In [ ]:
# run wilcox test for the motif scores
cd4na_dev_test <- cd4na_dev_score %>% filter(!is.na(deviation_score)) %>%
    group_by(motif) %>% t_test(data =., deviation_score ~ cohort, ref.group = 'Healthy') %>%
      adjust_pvalue(method = "BH") %>%
      add_significance("p.adj") 

In [ ]:
cd4na_dev_test %>% filter(str_detect(motif, 'NFAT'))

In [ ]:
# options(repr.plot.width = 20, repr.plot.height = 20)

p1 <- EnhancedVolcano::EnhancedVolcano(cd4na_dev_test, pointSize = 1,
                      title = NULL,
                      subtitle = NULL,
        lab = cd4na_dev_test %>% pull(motif),
        x = 'statistic',
        FCcutoff = 0.01, legendPosition = 'none',
        pCutoff = 0.05,
        y = 'p', labSize = 3.0) +
        coord_cartesian(xlim=c(-5, 5), ylim=c(0, 3)) + 
    ggthemes::theme_few()+ xlab('Divation score delta')+
        theme(legend.title=element_blank(), legend.position="none",
             text = element_text(size = 12))
p1
ggsave(file.path(fig_path, paste0(proj_name, '_cd4naive_TF_deviation_valcano_plot.png')), 
       width=6, height=6)

In [ ]:
# cd4na_dev_test %>% filter(p.adj<0.2)
cd4na_dev_mt <- cd4na_dev_test %>% filter(p<0.05)
cd4na_dev_mt %>% nrow()

In [ ]:
cd4na_dev_mx %>% hist()

In [ ]:
# make a matrix of the deviated motif
cd4na_dev_mx <- cd4na_dev_score %>% #filter(motif %in% cd4na_dev_mt$motif) %>% 
    pivot_wider(id_cols = 'motif', names_from = 'subject_id', values_from = 'deviation_score')
motif_name=cd4na_dev_mx$motif
cd4na_dev_mx <- cd4na_dev_mx %>% select_if(is.numeric) %>% as.matrix()
rownames(cd4na_dev_mx)=motif_name
cd4na_dev_mx %>% head()
cd4na_dev_mx = t(scale(t(cd4na_dev_mx)))
p1 <- ComplexHeatmap::Heatmap(cd4na_dev_mx,
                              col = circlize::colorRamp2(c(-2,0,2),
                                  c("purple",
                                    "black",
                                    "yellow")))

In [ ]:
png(file.path(fig_path, paste0(proj_name, '_cd4naive_TF_deviationheatmap.png')), units = 'in',res=300,
       width=8, height=12)
ComplexHeatmap::draw(p1)
dev.off()

In [ ]:
p1

In [ ]:
cd4na_dev_score %>% filter(motif %in% cd4na_dev_mt$motif) %>% 
    ggplot(aes(x=cohort, y=deviation_score))+
    geom_violin(aes(fill=cohort)) + 
    geom_jitter() + facet_wrap(vars(motif))
ggsave(file.path(fig_path, paste0(proj_name, '_chromVAR_deviation_sig_tf.pdf')),
       width = 12, height = 8)

In [ ]:
variability <- computeVariability(cd4na_dev)
plotVariability(variability, use_plotly = FALSE)

In [ ]:
colData(cd4na_dev)[, c('subject_id', 'cohort', 'age')]
sample_cor <- getSampleCorrelation(cd4na_dev)
sample_cor

In [ ]:
sample_cor <- getSampleCorrelation(cd4na_dev)

library(pheatmap)
my_sample_col <- colData(cd4na_dev)[, c('subject_id', 'cohort', 'age')] %>% as.data.frame()
pheatmap(as.dist(sample_cor), 
         annotation_col = my_sample_col, 
         annotation_row = my_sample_col, 
         clustering_distance_rows = as.dist(1-sample_cor), 
         clustering_distance_cols = as.dist(1-sample_cor))

In [ ]:
# tsne_results <- deviationsTsne(cd4na_dev, threshold = 1.5, perplexity = 10, 
#                                shiny = FALSE)
tsne_plots <- plotDeviationsTsne(cd4na_dev, tsne_results, annotation = c("STAT3", 'RORC'), 
                                   sample_column = "subject_id", shiny = FALSE)
tsne_plots

In [ ]:
diff_acc <- differentialDeviations(cd4na_dev, groups="cohort")

diff_acc %>% filter(p_value_adjusted<0.2)

#### check the motif deviation score vs the tf gene expression

In [ ]:
# check how many motif are present in the gene expression
table(rownames(cd4na_dev) %in% rownames(cd4na_pseudo))

In [ ]:
# take all the motif with overlap gene expression
overlap_mt <- unique(cd4na_dev_score$motif)[unique(cd4na_dev_score$motif) %in% rownames(cd4na_pseudo)]
overlap_mt %>% length()

In [ ]:
# get the gene expression of the motif
gex_mt <- assays(cd4na_pseudo)$logcounts[overlap_mt, ] %>% as_tibble(rownames = 'motif') %>%
    pivot_longer(cols = -motif, names_to = 'subject_id', values_to = 'gene_expression') %>%
    group_by(motif) %>% mutate(gene_expression_z = scale(gene_expression))
gex_mt %>% head()

In [ ]:
# combine the tf gene expression and motif deviation 
cd4na_mt_dev_gex <- cd4na_dev_score %>% filter(motif %in% gex_mt$motif) %>% 
    left_join(gex_mt, by=c('motif','subject_id')) %>% mutate(diff_sig=motif %in% cd4na_dev_mt)
cd4na_mt_dev_gex %>% head()

In [ ]:
# cd4na_mt_mx <- cd4na_mt_dev_gex %>% 

In [ ]:
cd4na_mt_dev_gex%>% ggplot(aes(x=deviation_score, y=gene_expression_z))+
    geom_point(aes(color=motif)) + theme(legend.position = "none")
ggsave(file.path(fig_path, paste0(proj_name, '_chromVAR_tf_deviation_gex_dotplot.pdf')), width = 8, height = 8)

In [ ]:
# # tf gene expression and tf deviation scores
# cd4na_mt_dev_gex %>% filter(diff_sig==TRUE)%>% ggplot(aes(x=deviation_score, y=gene_expression_z))+
#     geom_point(aes(color=subject_id)) + facet_wrap(vars(motif))+
#   geom_smooth(method='lm') + theme_bw()
# ggsave(file.path(fig_path, paste0(proj_name, '_chromVAR_tf_deviation_sig_gex.pdf')), width = 12, height = 8)

### motif to gene expressision correlation

In [ ]:
# load the motif matrix
cd4na_dev_score %>% head()

In [ ]:
# extract the deviation matrix
cd4na_dev_mx <- deviations(cd4na_dev)
colnames(cd4na_dev_mx) <- colData(cd4na_dev)$subject_id
cd4na_dev_mx <- cd4na_dev_mx[, order(colnames(cd4na_dev_mx))]

In [ ]:
cd4na_dev_mx %>% head()

In [ ]:
# load the pseudobulk gene matrix
ra_tea_l2_pseudo <- readRDS(file.path(data_path, 'PreRA_teaseq_nichenet_all_cells_l2_subject_pseudoBulk.rds'
                                ))
ra_tea_l2_pseudo

In [ ]:
# load the degs for cd4 naive
# load the deg list
ra_tea_l2_pseudo_degs <- 
    read_csv('/home/jupyter/data/preRA_teaseq/output_results/PreRA_teaseq_l2_annotation_PseudoBulk_degs_preRAvsHealthy.csv',
            show_col_types = FALSE) %>%
    filter(!is.na(FDR) & !is.na(logFC)) %>% mutate(direction=if_else(logFC>0, 'up', 'down'))
# filter cd4 naive
cd4na_pseudo_degs <- ra_tea_l2_pseudo_degs %>% filter(cell_type=='cd4_naive' & FDR<0.05)
cd4na_pseudo_degs %>% head()
cd4na_pseudo_degs %>% nrow()

In [ ]:
# cd4na_pseudo_degs %>% filter(str_detect(gene, 'IL'))

In [ ]:
# subset the cd4 naive
cd4na_pseudo <- ra_tea_l2_pseudo[cd4na_pseudo_degs$gene, 
                                 colData(ra_tea_l2_pseudo)$clean_l2_cell_types=='cd4_naive']
# make colnames of the pseudo object as subject id
colnames(cd4na_pseudo) <- colData(cd4na_pseudo)$subject_id
cd4na_pseudo

In [ ]:
# get sample level nomralized gene matrix from seurat object
ra_tea_cd4na <- SetIdent(ra_tea_cd4na, value = 'subject_id')
# get the sample average corrected SCT gene expression data
cd4na_gex_mx <- AverageExpression(ra_tea_cd4na, features = cd4na_pseudo_degs$gene, assays = 'SCT')$SCT

In [ ]:
cd4na_gex_mx %>% head()


In [ ]:
t(cd4na_dev_mx) %>% head()
t(cd4na_gex_mx) %>% head()

In [ ]:
motifs = colnames(t(cd4na_dev_mx))
gene_names <- colnames(t(cd4na_gex_mx))
x=t(cd4na_dev_mx)[, motifs[1]]
y=t(cd4na_gex_mx)[, gene_names[1]]
test <- jmuOutlier::perm.cor.test(x=x, y=y, 
                                  alternative = c("two.sided"), 
              method = c( "spearman"), num.sim = 20000)
test_res=tibble('motif'=motifs[1], 'gene'=gene_names[1], 'p.value'=test$p.value)

In [ ]:
motifs %>% length()

In [ ]:
# run permutation test of the motif to gene correlation
motifs = colnames(t(cd4na_dev_mx))
gene_names <- colnames(t(cd4na_gex_mx))

cd4na_dev_gex_cor_test <- lapply(motifs, function(motif){   
    test_res <- parallel::mclapply(gene_names, function(gene){
        x=t(cd4na_dev_mx)[, motif]
        y=t(cd4na_gex_mx)[, gene]
        test <- jmuOutlier::perm.cor.test(x=x, y=y, 
                                  alternative = c("two.sided"), 
              method = c("spearman"), num.sim = 20000)
        test_res=tibble('motif'=motif, 'gene'=gene, 'p.value'=test$p.value,
                       test="spearman")
        return(test_res)
    }, mc.cores=60) %>% rbindlist()
    return(test_res)
}) %>% rbindlist()


In [ ]:
cd4na_dev_gex_cor_test %>% head()

In [ ]:
cd4na_dev_gex_cor_test %>% distinct(motif) %>% nrow()
cd4na_dev_gex_cor_test %>% distinct(gene) %>% nrow()

In [ ]:
# library(corrr)
cd4na_dev_gex_cor <- cor(t(cd4na_dev_mx), t(cd4na_gex_mx), method = c("spearman"))
# symnum(clS <- cor(cd4na_dev_dex_cor, method = "spearman"))
cd4na_dev_gex_cor %>% dim()

In [ ]:
# load the nichnet gr network
cd4na_dev_gex_cor %>% hist()

In [ ]:
cd4na_dev_gex_cor %>% as_tibble(rownames = 'motif') %>% 
    pivot_longer(cols = -motif, names_to = 'gene', values_to = 'spearman_cor')

In [ ]:
# save the motif-gene correlation results
cd4na_tf_gex_cor <- cd4na_dev_gex_cor %>% as_tibble(rownames = 'motif') %>% 
    pivot_longer(cols = -motif, names_to = 'gene', values_to = 'spearman_cor') %>% 
    left_join(cd4na_dev_gex_cor_test, by=c('motif', 'gene'))
cd4na_tf_gex_cor %>%  write_tsv(file.path(output_path, 
                                  paste0(proj_name, '_mocha_ravshealthy_motif_gene_correlations_perm_test.tsv')))

In [ ]:
cd4na_tf_gex_cor <- read_tsv(file.path(output_path, 
                                  paste0(proj_name, '_mocha_ravshealthy_motif_gene_correlations_perm_test.tsv'))) 

cd4na_tf_gex_cor %>% head()

In [ ]:
cd4na_tf_gex_cor = cd4na_tf_gex_cor %>% filter(!is.na(p.value)) %>% 
    mutate(dev_sig=motif %in% (cd4na_dev_mt %>% filter(p<0.05) %>% pull(motif)))
cd4na_tf_gex_cor %>% head()

In [ ]:
# cd4na_tf_gex_cor %>% filter(!is.na(p.value)&dev_sig==TRUE)

In [ ]:
p1 <- cd4na_tf_gex_cor %>% filter(!is.na(p.value)) %>% ggplot(aes(x=spearman_cor, fill=dev_sig)) + 
    geom_histogram(alpha=0.6)
p2 <- cd4na_tf_gex_cor %>% filter(!is.na(p.value)) %>% ggplot(aes(x=p.value, fill=dev_sig)) + 
    geom_histogram(alpha=0.6) + geom_vline(xintercept=0.05,linetype="dashed", 
                color = "red")
cowplot::plot_grid(p1, p2)
ggsave(file.path(fig_path, paste0(proj_name, '_chromVAR_tf_deviation_gex_correlation_histogram_permutation_tese.png')),
       width = 12, height = 6)

In [ ]:
# join the correlation matrix with the permutation test results
sig_dev_motif = cd4na_dev_mt %>% filter(p<0.05) %>% pull(motif)
cd4na_dev_gex_cor_fl <- cd4na_tf_gex_cor %>% 
    filter(p.value<0.05& motif %in% sig_dev_motif) %>%
    mutate(corr_direction=if_else(spearman_cor>0, 'pos', 'neg'))
cd4na_dev_gex_cor_fl %>% head()

In [ ]:
motif_gene_cor_counts=cd4na_dev_gex_cor_fl %>% group_by(motif, corr_direction) %>% tally() %>%
    mutate(n=if_else(corr_direction=='neg', -n, n)) 
# motif_gene_cor_counts
motif_gene_cor_counts %>% ggplot(aes(x=n, y=motif, fill=corr_direction)) + geom_col()
ggsave(file.path(fig_path, paste0(proj_name, '_chromVAR_tf_deviation_gex_correlation_sig.png')),
       width = 6, height = 8)

In [ ]:
cd4na_dev_gex_cor_fl %>% distinct(motif) %>% nrow()
cd4na_dev_gex_cor_fl %>% distinct(gene) %>% nrow()
cd4na_dev_gex_cor_fl %>% group_by(motif) %>% tally() %>% arrange(desc(n))

In [ ]:
cd4na_dev_gex_cor_fl$spearman_cor %>% hist()

In [ ]:
cd4na_dev_gex_cor_fl

In [ ]:
### Read in NicheNet's tf-target prior model:
# validate the tf-target gene correlation
ref_path <- '/home/jupyter/data/Reference/Nichenet'
tf_target_matrix = readRDS(file.path(ref_path, "tf_target_matrix.rds"))

tf_target_matrix %>% head() # target genes in rows, tfs in columns

In [ ]:
tf_target_matrix <- tf_target_matrix %>% as_tibble(rownames = 'gene') %>% pivot_longer(cols = -gene, names_to = 'motif', values_to = 'nichenet_weight')

In [ ]:
cd4na_dev_gex_cor_fl <- cd4na_dev_gex_cor_fl %>% left_join(tf_target_matrix, by = c('motif', 'gene'))

In [ ]:
cd4na_dev_gex_cor_fl %>% arrange(desc(nichenet_weight))

In [ ]:
cd4na_dev_gex_cor_fl %>% mutate(motif_gene=paste(motif, gene, sep = '_')) %>%
    ggplot(aes(y=-log10(p.value), x=nichenet_weight)) +
    geom_point() + ggrepel::geom_label_repel(aes(label=motif_gene))
ggsave(file.path(fig_path, paste0(proj_name, '_chromVAR_tf_deviation_gex_correlation_nichnet.png')),
       width = 8, height = 8)

In [ ]:
p1 <- cd4na_dev_gex_cor_fl %>% ggplot(aes(x=nichenet_weight)) +geom_histogram()
p2 <- tf_target_matrix%>% ggplot(aes(x=nichenet_weight)) +geom_histogram()
p1+p2


In [ ]:
enrich_cd4na_tf_deg <- enrichr(cd4na_dev_gex_cor_fl$gene, c('KEGG_2021_Human', 'MSigDB_Hallmark_2020'))


In [ ]:
enrich_cd4na_tf_deg$MSigDB_Hallmark_2020 %>% filter(Adjusted.P.value<0.05) %>% select(-c(Old.P.value,Old.Adjusted.P.value))
oxphos_gene = enrich_cd4na_tf_deg$MSigDB_Hallmark_2020 %>% filter(Adjusted.P.value<0.05&Term=='Oxidative Phosphorylation') %>%
    pull(Genes) %>% str_split(';', simplify = TRUE)
nfkb_gene = enrich_cd4na_tf_deg$MSigDB_Hallmark_2020 %>% filter(Adjusted.P.value<0.05&Term=='TNF-alpha Signaling via NF-kB') %>%
    pull(Genes) %>% str_split(';', simplify = TRUE)
il2_gene = enrich_cd4na_tf_deg$MSigDB_Hallmark_2020 %>% filter(Adjusted.P.value<0.05&Term=='IL-2/STAT5 Signaling') %>%
    pull(Genes) %>% str_split(';', simplify = TRUE)

In [ ]:
cd4na_dev_gex_cor_oxphos <- cd4na_dev_gex_cor_fl %>% filter(gene %in% il2_gene)
cd4na_dev_gex_cor_oxphos %>% head()

In [ ]:
cd4na_dev_gex_cor_oxphos %>% arrange(desc(nichenet_weight)) %>% head()

In [ ]:
cd4na_dev_gex_cor_fl_mx <- cd4na_dev_gex_cor_oxphos %>% 
    pivot_wider(id_cols = 'motif', names_from = 'gene', values_from = 'spearman_cor',values_fill = 0)
cd4na_dev_gex_cor_fl_matrix=cd4na_dev_gex_cor_fl_mx %>%select_if(is.numeric) %>%as.matrix()
rownames(cd4na_dev_gex_cor_fl_matrix)=cd4na_dev_gex_cor_fl_mx$motif
# cd4na_dev_gex_cor_fl_mx[cd4na_dev_gex_cor_fl_mx<0.8] = 0

In [ ]:
cd4na_dev_gex_cor_fl_matrix %>% head()

In [ ]:
# cd4na_dev_gex_cor_fl_mx <- cd4na_dev_gex_cor[unique(cd4na_dev_gex_cor_fl$motif), 
#                                              unique(cd4na_dev_gex_cor_fl$gene)] %>% t()
# # filter 
# cd4na_dev_gex_cor_fl_mx <- cd4na_dev_gex_cor_fl_mx[if_else(cd4na_dev_gex_cor_fl_mx<0.8, 0, cd4na_dev_gex_cor_fl_mx)]
# cd4na_dev_gex_cor_fl_mx %>%dim()
motif_sig <- data.frame(motif_sig=ifelse(colnames(cd4na_dev_gex_cor_fl_mx) %in% cd4na_dev_mt,
                                         "Sig", "Not Sig"))

p1 <- ComplexHeatmap::Heatmap(cd4na_dev_gex_cor_fl_matrix %>% t(), show_row_names=TRUE,
                              show_column_names = TRUE, row_title = 'Motif', column_title = 'Gene')

pdf(file.path(fig_path, paste(proj_name, 'motif_deviatoin_deg_sig_expression_spearman_cor_heatmap_il2_gene.pdf', sep='_')), 
               width=16, height=12)
p1
dev.off()  


In [ ]:
p1

### check what are the overlap of genes and atac openness

In [ ]:
# load the degs for cd4 naive
# load the deg list
ra_tea_l2_pseudo_degs <- 
    read_csv('/home/jupyter/data/preRA_teaseq/output_results/PreRA_teaseq_l2_annotation_PseudoBulk_degs_preRAvsHealthy.csv',
            show_col_types = FALSE) %>%
    filter(!is.na(FDR) & !is.na(logFC)) %>% mutate(direction=if_else(logFC>0, 'up', 'down'))
# filter cd4 naive
cd4na_pseudo_degs <- ra_tea_l2_pseudo_degs %>% filter(cell_type=='cd4_naive')
cd4na_pseudo_degs %>% head()

In [ ]:
cd4na_enrich_mt <- differentials_motif_ravshealthy %>% filter(adjp_val<0.05) %>% pull(motif)

In [ ]:
# filter out the differential peaks
cd4na_diff_peaks <- differentials %>% filter(FDR<0.2&!is.na(Gene)) %>% dplyr::rename('gene'='Gene')

cd4na_diff_peaks_unique <- cd4na_diff_peaks %>% arrange(FDR) %>% distinct(gene, .keep_all = TRUE)
table(cd4na_diff_peaks_unique$gene %in% cd4na_pseudo_degs$gene)

In [ ]:
cd4na_diff_peaks %>% janitor::get_dupes(gene)

In [ ]:
# combine the degs and deps
cd4na_pseudo_degs_deps <- cd4na_pseudo_degs %>% filter(gene %in% cd4na_diff_peaks_unique$gene) %>% 
    left_join(cd4na_diff_peaks_unique, suffix = c('.degs', '.deps'), by='gene')
cd4na_pseudo_degs_deps

In [ ]:
cd4na_pseudo_degs_deps %>% ggplot() + geom_point(aes(x=logFC, y=MeanDiff, color=tileType))

# TF activity prediction

## construct nichenet context specific models

In [ ]:
# load the nichenet prior network
ref_path <- '/home/jupyter/data/Reference/Nichenet'

In [ ]:
# load the deg list
ra_tea_l2_pseudo_degs <- read_csv(
    '/home/jupyter/data/preRA_teaseq/output_results/PreRA_teaseq_l2_annotation_PseudoBulk_degs_preRAvsHealthy.csv',
    show_col_types = FALSE)
ra_tea_l2_pseudo_degs %>% head()

In [ ]:
# filter degs for cd4 naive
cd4na_degs <- ra_tea_l2_pseudo_degs %>% filter(cell_type=='cd4_naive' & FDR<0.05)
cd4na_degs %>% dim()

In [ ]:
### Read in NicheNet's tf-target prior model:
tf_target_matrix = readRDS(file.path(ref_path, "tf_target_matrix.rds"))

tf_target_matrix[1:5,1:5] # target genes in rows, tfs in columns

In [ ]:
tf_target_matrix %>% dim()

In [ ]:
# filter by any motif present in the atac data
motif_present <- metadata(SampleTileMatricesAnnotated)$Motifs %>% names()
motif_present %>% length()
table(motif_present %in% colnames(tf_target_matrix))

In [ ]:
# filter the motif that are present in the data
tf_target_matrix_fl <- tf_target_matrix[, colnames(tf_target_matrix) %in% motif_present]

In [ ]:
table(unique(gr_network$from) %in% colnames(tf_target_matrix_fl))
table(unique(gr_network$to) %in% rownames(tf_target_matrix_fl))

In [ ]:

colnames(tf_target_matrix)%>% str_subset('STAT')

In [ ]:
extract_top_n_targets("STAT3", 5, tf_target_matrix)

### define receiver cells

In [ ]:
ra_tea_cd4na@meta.data %>% colnames()

In [ ]:
# define figname to save all nichenet figs as sender to receiver
figname='tf_cd4_naive'

In [ ]:
# define all the innate cell types
receiver_ct <- ra_tea_cd4na@meta.data %>% distinct(t_annotation) %>% 
    pull(t_annotation) 
receiver_ct

In [ ]:
## receiver
ra_tea_cd4na <- SetIdent(ra_tea_cd4na, value = 't_annotation')
receiver = receiver_ct
expressed_genes_receiver = get_expressed_genes(receiver, ra_tea_cd4na, pct = 0.10)
background_expressed_genes = expressed_genes_receiver %>% .[. %in% rownames(tf_target_matrix)]

In [ ]:
# Define a gene set of interest
geneset_oi = cd4na_degs %>% pull(gene) %>% .[. %in% rownames(tf_target_matrix)]
geneset_oi %>% length()
table(pull(cd4na_degs, gene) %in% rownames(tf_target_matrix))

In [ ]:
cd4na_degs %>% filter(!gene %in% rownames(tf_target_matrix)) %>% pull(gene)

In [ ]:
# Define a set of potential tfs
tfs = colnames(tf_target_matrix)
length(tfs)
# check what tf are expressed and 
expressed_tfs = intersect(tfs,expressed_genes_receiver)
motif_present_tf = intersect(tfs, names(metadata(SampleTileMatricesAnnotated)$Motifs))

table(expressed_tfs %in% tfs)
table(motif_present_tf %in% tfs)
potential_tfs = unique(append(motif_present_tf, expressed_tfs))

In [ ]:
potential_tfs %>% length()

In [ ]:
#Perform NicheNet tf activity analysis: rank the potential tfs 
# based on the presence of their target genes in the gene set of interest (compared to the background set of genes)

tf_activities = suppressWarnings(predict_ligand_activities(geneset = geneset_oi, 
                                                           background_expressed_genes = background_expressed_genes, 
                                          ligand_target_matrix = tf_target_matrix, potential_ligands = potential_tfs))

# tf_activities = tf_activities %>% arrange(-pearson) %>% mutate(rank = rank(desc(pearson)))
# tf_activities

In [ ]:
tf_activities <- tf_activities %>% filter(!is.na(pearson))%>% 
    arrange(-pearson) %>% mutate(rank = rank(dplyr::desc(pearson))) %>%
    mutate(tf_expressed=(test_ligand %in% expressed_tfs), 
          motif_present=(test_ligand %in% motif_present_tf))
tf_activities %>% write_csv(file.path(output_path, paste0(proj_name,figname, '_tf_prediction_degs.csv')))

In [ ]:
potential_tfs %>% length()

In [ ]:
tf_activities %>% head(10)

In [ ]:
best_upstream_tfs = bind_rows(tf_activities %>% top_n(20, pearson) %>% arrange(-pearson) , 
                      tf_activities %>% top_n(20, auroc) %>% 
                              arrange(-auroc)) %>% distinct(test_ligand, .keep_all = TRUE)
best_upstream_tfs

In [ ]:
active_tf_target_links_df = best_upstream_tfs$test_ligand %>% lapply(get_weighted_ligand_target_links,
                                                         geneset = geneset_oi, 
                                                         ligand_target_matrix = tf_target_matrix, n = 100) %>% 
    bind_rows() %>% drop_na()

active_tf_target_links = prepare_ligand_target_visualization(ligand_target_df = active_tf_target_links_df,
                                                             ligand_target_matrix = tf_target_matrix, cutoff = 0.2)
active_tf_target_links %>% head()
order_tfs = intersect(best_upstream_tfs, colnames(active_tf_target_links)) %>% rev() %>% make.names()
order_targets = active_tf_target_links_df$target %>% unique() %>% intersect(rownames(active_tf_target_links)) %>% make.names()
rownames(active_tf_target_links) = rownames(active_tf_target_links) %>% make.names() # make.names() for heatmap visualization of genes like H2-T23
colnames(active_tf_target_links) = colnames(active_tf_target_links) %>% make.names() # make.names() for heatmap visualization of genes like H2-T23

vis_tf_target = active_tf_target_links[order_targets,order_tfs] %>% t()

In [ ]:
geneset_oi %>% length()

In [ ]:
active_tf_target_links_df

In [ ]:
quiet_library('nichenetr')

In [ ]:
# load the nichenet ligand target database
nichenet_path <- '/home/jupyter/github/nichenetr/data'

In [ ]:
# load the gene regulatory network
# this the tf to gene connection
gr_network = readRDS(file.path(nichenet_path, "gr_network.rds"))
gr_network %>% head()

In [ ]:
motif_sets <- metadata(SampleTileMatricesAnnotated)$Motifs %>% names()
table(motif_sets %in% unique(gr_network$from))

In [ ]:
metadata(SampleTileMatricesAnnotated)$Motifs

In [ ]:
motif_sets %>% length()

In [ ]:
### Read in NicheNet's tf-target prior model:
tf_target_matrix = readRDS(file.path('/home/jupyter/data/Reference/Nichenet',
                                     "tf_target_matrix.rds"))
tf_target_matrix %>% head()

In [ ]:
table(rownames(tf_target_matrix) %in% names(metadata(SampleTileMatricesAnnotated)$Motifs))
table(names(metadata(SampleTileMatricesAnnotated)$Motifs) %in% rownames(tf_target_matrix))

In [ ]:
tf_target_matrix[rownames(tf_target_matrix) %in% names(metadata(SampleTileMatricesAnnotated)$Motifs), ] %>% hist()

In [ ]:
# load the tf prediction list
nichenet_tf <- read_csv('/home/jupyter/data/preRA_teaseq/output_results/nichenet/tf_cd4_naive_tf_prediction_degs.csv',
                       show_col_types = FALSE) 
table(nichenet_tf$test_ligand %in% differentials_motif_ravshealthy$motif)

nichenet_tf_fl <- nichenet_tf %>% filter(test_ligand %in% differentials_motif_ravshealthy$motif) %>% 
    dplyr::rename('motif'='test_ligand')
nichenet_tf_fl%>% head(20)

In [ ]:
differentials_motif_ravshealthy %>%
    filter(adjp_val<0.2) %>% 
    left_join(nichenet_tf_fl, by = 'motif') %>% arrange(rank)

In [ ]:
# define figname to save all nichenet figs as sender to receiver
figname='tf_cd4_naive'

In [ ]:
# define all the innate cell types
receiver_ct <- ra_tea_cd4na@meta.data %>% distinct(t_annotation) %>% 
    pull(t_annotation)
receiver_ct

In [ ]:
## receiver
ra_tea_cd4na <- SetIdent(ra_tea_cd4na, value = 't_annotation')
receiver = receiver_ct
expressed_genes_receiver = get_expressed_genes(receiver, ra_tea_cd4na, pct = 0.10)
background_expressed_genes = expressed_genes_receiver %>% .[. %in% rownames(tf_target_matrix)]

In [ ]:
background_expressed_genes %>% length()

In [ ]:
cd4na_degs <- ra_tea_l2_pseudo_degs %>% filter(cell_type=='cd4_naive'&FDR<0.2)

In [ ]:
# Define a gene set of interest
geneset_oi = cd4na_degs %>% pull(gene) %>% .[. %in% rownames(tf_target_matrix)]
geneset_oi %>% length()
table(pull(cd4na_degs, gene) %in% rownames(tf_target_matrix))

In [ ]:
# filter TF matrix by the presented tf
tf_target_matrix_fl <- tf_target_matrix[rownames(tf_target_matrix) %in% names(metadata(SampleTileMatricesAnnotated)$Motifs), ]
tf_target_matrix_fl %>% dim()

In [ ]:
# Define a set of potential tfs
tfs = colnames(tf_target_matrix_fl)
expressed_tfs = intersect(tfs,expressed_genes_receiver)
potential_tfs = tfs
table(tfs %in% expressed_tfs)

In [ ]:
#Perform NicheNet tf activity analysis: rank the potential tfs 
# based on the presence of their target genes in the gene set of interest (compared to the background set of genes)

tf_activities = suppressWarnings(predict_ligand_activities(geneset = geneset_oi, background_expressed_genes = background_expressed_genes, 
                                          ligand_target_matrix = tf_target_matrix_fl, potential_ligands = potential_tfs))

# tf_activities = tf_activities %>% arrange(-pearson) %>% mutate(rank = rank(desc(pearson)))
# tf_activities

In [ ]:
tf_activities <- tf_activities %>% filter(!is.na(pearson))%>% 
    arrange(-pearson) %>% mutate(rank = rank(dplyr::desc(pearson)))
tf_activities %>% write_csv(file.path(output_path, paste0(figname, '_tf_prediction_degs_fl_atac.csv')))

In [ ]:
tf_activities %>% head(10)

In [ ]:
best_upstream_tfs = bind_rows(tf_activities %>% top_n(20, pearson) %>% arrange(-pearson) , 
                      tf_activities %>% top_n(20, auroc) %>% arrange(-auroc)) %>% distinct(test_ligand, .keep_all = TRUE)
best_upstream_tfs

In [ ]:
active_tf_target_links_df = best_upstream_tfs %>% lapply(get_weighted_ligand_target_links,
                                                         geneset = geneset_oi, 
                                                         ligand_target_matrix = tf_target_matrix_fl, n = 100) %>% bind_rows() %>% drop_na()

active_tf_target_links = prepare_ligand_target_visualization(ligand_target_df = active_tf_target_links_df, ligand_target_matrix = tf_target_matrix_fl, cutoff = 0.33)

order_tfs = intersect(best_upstream_tfs, colnames(active_tf_target_links)) %>% rev() %>% make.names()
order_targets = active_tf_target_links_df$target %>% unique() %>% intersect(rownames(active_tf_target_links)) %>% make.names()
rownames(active_tf_target_links) = rownames(active_tf_target_links) %>% make.names() # make.names() for heatmap visualization of genes like H2-T23
colnames(active_tf_target_links) = colnames(active_tf_target_links) %>% make.names() # make.names() for heatmap visualization of genes like H2-T23

vis_tf_target = active_tf_target_links[order_targets,order_tfs] %>% t()

# import the peaks cells in scMACS into Archr and remake peak matrix

In [ ]:
# Tests if a string is a in the correct format to convert to GRanges 
validRegionString <- function(regionString) {
  if (!is.character(regionString)) {
    return(FALSE)
  }

  pattern <- "([0-9]{1,2}|chr[0-9]{1,2}):[0-9]*-[0-9]*"
  matchedPattern <- str_extract(regionString, pattern)

  if (is.na(matchedPattern)) {
    return(FALSE)
  } else if (!matchedPattern == regionString) {
    return(FALSE)
  }

  splits <- str_split(regionString, "[:-]")[[1]]
  start <- splits[2]
  end <- splits[3]
  if (start > end) {
    return(FALSE)
  }

  # All conditions satisfied
  return(TRUE)
}
# Strings to GRanges
StringsToGRanges <- function(regionString) {
  if (!validRegionString(regionString)) {
    stop("Region must be a string matching format 'seqname:start-end', where start<end e.g. chr1:123000-123500")
  }

  chrom <- gsub(":.*", "", regionString)
  startSite <- gsub(".*:", "", regionString) %>%
    gsub("-.*", "", .) %>%
    as.numeric()
  endSite <- gsub(".*-", "", regionString) %>% as.numeric()


  regionGRanges <- GRanges(seqnames = chrom, ranges = IRanges(start = startSite, end = endSite), strand = "*")
  return(regionGRanges)
}  

In [ ]:
SampleTileMatricesAnnotated %>% rowData()

In [ ]:
# make a grange object from the the union of the peaks called
cd4na_preRAvsHealthy_peaks <- SampleTileMatricesAnnotated %>% rownames() %>% unique() %>% 
    StringsToGRanges()
cd4na_preRAvsHealthy_peaks

In [ ]:
# add the peak called by scMACS back to archR
cd4na_atac <- addPeakSet(
  ArchRProj = cd4na_atac,
  peakSet = cd4na_preRAvsHealthy_peaks,
  #genomeAnnotation = getGenomeAnnotation(cd4na_atac),
  force = TRUE
)


In [ ]:
# add peakmatrix based on the peak set
cd4na_atac <- addPeakMatrix(cd4na_atac)
getAvailableMatrices(cd4na_atac)

In [ ]:
# rerun a lsi based on the peakmatrix
# add lsi in atac
cd4na_atac <- addIterativeLSI(
      ArchRProj = cd4na_atac,
      useMatrix = "PeakMatrix", 
      name = "IterativeLSI_peak", 
      iterations = 2,
      #varFeatures = 75000, # increase the viable features
      force = TRUE
    )

In [ ]:
# redo clustering based on the lsi from scMACS called peak matrix
cd4na_atac <- addClusters(
    input = cd4na_atac,
    reducedDims = "IterativeLSI_peak",
    method = "Seurat",
    name = "peak_Clusters_0.5",
    resolution = 0.5,
    force = TRUE
)

In [ ]:
cd4na_atac <- addUMAP(
    ArchRProj = cd4na_atac, 
    reducedDims = "IterativeLSI_peak", 
    name = "UMAP_peak", 
    nNeighbors = 30, 
    minDist = 0.5, 
    metric = "cosine",
    force = TRUE
)

In [ ]:
saveArchRProject(ArchRProj = cd4na_atac)

In [ ]:
p1 <- cd4na_atac %>% BoxPlotClusterFreq(cluster.name='peak_Clusters_0.5',
                                        totalcounts = 'total_counts',
                                          color.by='cohort', log.y = FALSE)
p1 

In [ ]:
# plot umap of annotated cell types
p1 <- plotEmbedding(cd4na_atac, embedding = "UMAP_peak", 
                    colorBy = "cellColData", name = c("l2_cell_types", "peak_Clusters_0.5",'peak_Clusters_0.8',
                                                      'subject_id','age','predicted.celltype_l2','wsnn_res.0.5',
                                                      'cohort'))
plotPDF(p1, name = paste0(proj_name, '_Clusters_0.8_atac_peak_umap_gene_score.pdf'), ArchRProj = cd4na_atac,
        addDOC = FALSE, width = 6, height = 6)

In [ ]:
# plot density UMAP
plotDensity(ArchRProj= cd4na_atac, embeddingName= 'UMAP_peak', identity = 'cohort', returnObj = FALSE)

In [ ]:
cd4na_atac$peak_Clusters_0.5 %>% table()
cd4na_atac$cohort %>% unique()

In [ ]:
# Identifying Marker Peaks with ArchR
cd4na_ravshealthy_Peaks <- getMarkerFeatures(
    ArchRProj = cd4na_atac, 
    useMatrix = "PeakMatrix", 
    useGroups = c('C4', 'C5'),
   #bgdGroups = c('C1', 'C2','C3', 'C6'),
    groupBy = "peak_Clusters_0.5",
  bias = c("TSSEnrichment", "log10(nFrags)"),
     maxCells = 2500,
  testMethod = "wilcoxon"
)

In [ ]:
markerList <- getMarkers(cd4na_ravshealthy_Peaks, cutOff = "FDR <= 0.01 & Log2FC >= 1")
markerList

In [ ]:
# plot the heatmap of differential peaks
heatmapPeaks <- plotMarkerHeatmap(
  seMarker = cd4na_ravshealthy_Peaks, 
  cutOff = "FDR <= 0.05 & abs(Log2FC) >= 1",
    plotLog2FC = TRUE,
  transpose = TRUE
)
draw(heatmapPeaks, heatmap_legend_side = "bot", annotation_legend_side = "bot")
plotPDF(heatmapPeaks, name = "cd4na_ravshealthy_Peaks_Marker-Heatmap", 
        width = 8, height = 6, ArchRProj = cd4na_atac, addDOC = FALSE)

In [ ]:
# Motif Enrichment in Differential Peaks

# cisbp lower quality higher coverage
# japstat higher quality lower coverage
# cd4na_atac <- addMotifAnnotations(ArchRProj = cd4na_atac, motifSet = "cisbp", name = "Motif",
#                                  force = TRUE)

In [ ]:
# identify the enriched motif for all clutsers
cd4na_ravshealthy_Peaks_motif <- peakAnnoEnrichment(
    seMarker = cd4na_ravshealthy_Peaks,
    ArchRProj = cd4na_atac,
    peakAnnotation = "Motif",
    cutOff = "FDR <= 0.2 & Log2FC >= 0.5"
  )

In [ ]:
rownames(cd4na_ravshealthy_Peaks_motif) %>% str_subset('STAT')

In [ ]:
heatmapEM <- plotEnrichHeatmap(cd4na_ravshealthy_Peaks_motif, n = 50, transpose = TRUE)
ComplexHeatmap::draw(heatmapEM, heatmap_legend_side = "bot", annotation_legend_side = "bot")
plotPDF(heatmapEM, name = "cd4na_ravshealthy_Peaks_motif_Peaks_Motif-Heatmap", 
        width = 8, height = 6, ArchRProj = cd4na_atac, addDOC = FALSE)

In [ ]:
# identify the enriched motif
# exlcuding C4
cd4na_clusters_0.5_noc4_motif <- peakAnnoEnrichment(
    seMarker = cd4na_clusters_0.5_c4_Peaks[,c(1:3, 5, 6)],
    ArchRProj = cd4na_atac,
    peakAnnotation = "Motif",
    cutOff = "FDR <= 0.1 & Log2FC >= 0.5"
  )

In [ ]:
cd4na_clusters_0.5_motif[,c(1:3, 5, 6)]

In [ ]:
heatmapEM <- plotEnrichHeatmap(cd4na_clusters_0.5_noc4_motif, n = 50, transpose = TRUE)
ComplexHeatmap::draw(heatmapEM, heatmap_legend_side = "bot", annotation_legend_side = "bot")
plotPDF(heatmapEM, name = "cd4na_clusters_0.5_noC4_Peaks_Motif-Heatmap", 
        width = 8, height = 6, ArchRProj = cd4na_atac, addDOC = FALSE)

### motif enrichment by cell types

In [ ]:
# Motif Enrichment in Differential Peaks

# cisbp lower quality higher coverage
# japstat higher quality lower coverage
cd4na_atac <- addMotifAnnotations(ArchRProj = cd4na_atac, motifSet = "cisbp", name = "Motif",
                                 force = TRUE)

In [ ]:
cd4na_atac

In [ ]:
# identify the enriched motif
motifsEnriched <- peakAnnoEnrichment(
    seMarker = cd4na_clusters_0.5_Peaks,
    ArchRProj = cd4na_atac,
    peakAnnotation = "Motif",
    cutOff = "FDR <= 0.1 & abs(Log2FC) >= 0.5"
  )

In [ ]:
motifsEnriched

In [ ]:
heatmapEM <- plotEnrichHeatmap(motifsEnriched, n = 100, transpose = TRUE)
ComplexHeatmap::draw(heatmapEM, heatmap_legend_side = "bot", annotation_legend_side = "bot")
plotPDF(heatmapEM, name = "cd4na_clusters_0.5_Peaks_Motif-Heatmap", 
        width = 8, height = 6, ArchRProj = cd4na_atac, addDOC = FALSE)

In [ ]:
# add Encode TF Binding Sites
cd4na_atac <- addArchRAnnotations(ArchRProj = cd4na_atac, force = TRUE,
                                  collection = "EncodeTFBS")

In [ ]:
cd4na_clusters_0.5_Peaks

In [ ]:
enrichEncode <- peakAnnoEnrichment(
    seMarker = cd4na_clusters_0.5_Peaks,
    ArchRProj = cd4na_atac,
    peakAnnotation = "EncodeTFBS",
    cutOff = "FDR <= 0.1 & abs(Log2FC) >= 0.5"
  )

In [ ]:
heatmapEncode <- plotEnrichHeatmap(enrichEncode, n = 20, transpose = TRUE)
ComplexHeatmap::draw(heatmapEncode, heatmap_legend_side = "bot", annotation_legend_side = "bot")
plotPDF(heatmapEncode, name = "EncodeTFBS-Enriched-cd4na_clusters_0.5_Markers-Heatmap", width = 8, height = 6, 
        ArchRProj = cd4na_atac, addDOC = FALSE)

### Motif Deviations with chromvar

In [ ]:
# Motif Deviations
cd4na_atac <- addBgdPeaks(cd4na_atac )

In [ ]:
cd4na_atac <- addDeviationsMatrix(
  ArchRProj = cd4na_atac, 
    peakAnnotation='MochaPeakMatrix',
  force = TRUE
)

In [ ]:
getAvailableMatrices(cd4na_atac)

In [ ]:
plotVarDev <- getVarDeviations(cd4na_atac, name = "MotifMatrix", plot = TRUE)
plotVarDev
plotPDF(plotVarDev, name = "Variable-Motif-Deviation-Scores", 
        width = 12, height = 12, ArchRProj = cd4na_atac, addDOC = FALSE)

In [ ]:
cd4na_dev_mt

In [ ]:
# motifs <- c("GATA1", "CEBPA", "EBF1", "IRF4", "TBX21", "PAX5")
markerMotifs <- getFeatures(cd4na_atac, select = paste(cd4na_dev_mt, collapse="|"), useMatrix = "MotifMatrix")
markerMotifs <- grep("z:", markerMotifs, value = TRUE)
markerMotifs

In [ ]:
markerMotifs %in% getFeatures(cd4na_atac, 'MotifMatrix')

In [ ]:
p <- plotEmbedding(
    ArchRProj = cd4na_atac, 
    colorBy = "MotifMatrix", 
    name = sort(markerMotifs), 
    embedding = "UMAP",
    imputeWeights = getImputeWeights(cd4na_atac)
)

In [ ]:
p

In [ ]:
# rerun a lsi based on the peakmatrix
# add lsi in atac
cd4na_atac <- addIterativeLSI(
      ArchRProj = cd4na_atac,
      useMatrix = "MotifMatrix", 
      name = "IterativeLSI_motif", 
      iterations = 2,
      #varFeatures = 75000, # increase the viable features
      force = TRUE
    )

cd4na_atac <- addUMAP(
    ArchRProj = cd4na_atac, 
    reducedDims = "IterativeLSI_motif", 
    name = "UMAP_motif", 
    nNeighbors = 30, 
    minDist = 0.5, 
    metric = "cosine",
    force = TRUE
)

In [ ]:
saveArchRProject(ArchRProj = cd4na_atac)

### peak2gene linkage

In [ ]:
getAvailableMatrices(cd4na_atac)

In [ ]:
cd4na_atac <- addPeak2GeneLinks(
    ArchRProj = cd4na_atac,
    useMatrix = "GeneExpressionMatrix",
    reducedDims = "IterativeLSI"
)

In [ ]:
p2g <- getPeak2GeneLinks(
    ArchRProj = cd4na_atac,
    corCutOff = 0.45,
    resolution = 1,
    returnLoops = TRUE
)

In [ ]:
p2g[[1]]

### plot regions in Archr 

In [ ]:
source('/home/jupyter/github/scATAC_Supplements/ArchR_Export_ATAC_Data.R')

In [ ]:
# Extract fragments by populations from an ArchR Project: getPopFrags()
cd4na_Clusters_0.5_frags <- getPopFrags(cd4na_atac, 'Clusters_0.5',cellSubsets = 'ALL' ,
                                           region = NULL, numCores = 30, 
                        NormMethod = "nfrags", blackList = NULL, overlapList = 50)

In [ ]:
cd4na_Clusters_0.5_frags_files <- FragsToCoverageFiles(cd4na_Clusters_0.5_frags, files = "BigWig",
                                             genome = "hg38", fname = NULL, outDir = NULL, numCores=30)

In [ ]:
getAvailableMatrices(cd4na_atac)
cd4na_atac <- addMotifAnnotations(ArchRProj = cd4na_atac, motifSet = "JASPAR2020", name = "JasparMotifs")

In [ ]:
getAvailableMatrices(cd4na_atac)

In [ ]:
cd4na_Clusters_0.5_frags_PLCG2 <- getbpCounts(regionString='chr12:68,154,768-68,159,740', 
                                          popFrags=cd4na_Clusters_0.5_frags,
                                        numCores = 30, returnGRanges = FALSE)

In [ ]:
cd4na_Clusters_0.5_frags_PLCG2 %>% head()

In [ ]:
suppressMessages(p1 <- plotRegion(cd4na_Clusters_0.5_frags_PLCG2, 
                                  ArchRProj = cd4na_atac, motifSetName =  "JasparMotifs"))


In [ ]:
pdf(file.path(fig_path, paste0(proj_name, '_atac_Clusters_0.8_IFNG_gene_coverage.pdf')), width = 12, height = 12)
suppressWarnings(p1)
dev.off()

In [ ]:
p1

In [ ]:
sessionInfo()

### (archive) call peaks in the archR (MACS2)

In [ ]:
# add group coverage
cd4na_atac <- addGroupCoverages(ArchRProj = cd4na_atac, maxReplicates = 16,
                                 groupBy = "Clusters_0.5")

In [ ]:
# !!important
# define the path in conda environment to mac2 python packages for peak calling in archr
pathToMacs2 <- '/home/jupyter/libs/r_scrna/bin/macs2'
pathToMacs2

In [ ]:
# call peaks in cd4 naive
cd4na_atac <- addReproduciblePeakSet(threads = 60,
    ArchRProj = cd4na_atac, 
    groupBy = "Clusters_0.5", 
    pathToMacs2 = pathToMacs2,
    force = TRUE
)

In [ ]:
cd4na_atac <- addPeakMatrix(cd4na_atac)

In [ ]:
getAvailableMatrices(cd4na_atac)

In [ ]:
saveArchRProject(ArchRProj = cd4na_atac)

In [ ]:
table(cd4na_atac$Clusters_0.5)

In [ ]:
#Identifying Marker Peaks with ArchR
cd4na_clusters_0.5_Peaks <- getMarkerFeatures(
    ArchRProj = cd4na_atac, 
    useMatrix = "PeakMatrix", 
    groupBy = "Clusters_0.5",
     maxCells = 1135,
  bias = c("TSSEnrichment", "log10(nFrags)"),
  testMethod = "wilcoxon"
)

In [ ]:
markerList <- getMarkers(cd4na_clusters_0.5_Peaks, cutOff = "FDR <= 0.01 & Log2FC >= 1")
markerList

In [ ]:
# plot the heatmap of differential peaks
heatmapPeaks <- plotMarkerHeatmap(
  seMarker = cd4na_clusters_0.5_Peaks, 
  cutOff = "FDR <= 0.1 & Log2FC >= 0.5",
  transpose = TRUE
)
draw(heatmapPeaks, heatmap_legend_side = "bot", annotation_legend_side = "bot")
plotPDF(heatmapPeaks, name = "cd4na_clusters_0.5_Peaks_Marker-Heatmap", 
        width = 8, height = 6, ArchRProj = cd4na_atac, addDOC = FALSE)

In [ ]:
sessionInfo()